In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np

In [2]:
# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
#from code.loading import DataLoader
from src.utils.statistic import Chi2Test

In [3]:
# lf = pl.read_csv(
#     "maude_extracted_sample2.csv",
#     infer_schema_length=0
# ) # 일단 모든 data 문자열로 받음

In [4]:
# scan_csv로 LazyFrame 직접 생성 (메모리에 로드 안 함)
lf = pl.scan_parquet(
    "/Users/t2023-m0030/Desktop/i_hate_maude/notebooks/maude_10000_2025-12-19_2.parquet"
    # ,infer_schema_length=0
    # ,ignore_errors=True
)

In [5]:
total_rows = lf.select(pl.len()).collect().item()
total_cols = len(lf.collect_schema().names())
print(f"전체 행: {total_rows:,}개, 전체 컬럼: {total_cols}개")

전체 행: 10,000개, 전체 컬럼: 26개


In [6]:
lf.schema

/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_56394/2177650233.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  lf.schema


Schema([('mdr_report_key', Int32),
        ('adverse_event_flag', Boolean),
        ('product_problem_flag', Boolean),
        ('date_of_event', Date),
        ('date_received', Date),
        ('device_date_of_manufacturer', Date),
        ('event_type', Categorical),
        ('previous_use_code', Boolean),
        ('single_use_flag', Boolean),
        ('reprocessed_and_reused_flag', Boolean),
        ('product_problems', String),
        ('manufacturer', String),
        ('brand', String),
        ('model_number', String),
        ('udi_di', String),
        ('product_code', String),
        ('operator', Boolean),
        ('device_name', String),
        ('patient_age', Int64),
        ('mdr_text', String),
        ('patient_harm', String),
        ('problem_components', List(String)),
        ('incident_summary', String),
        ('defect_confirmed', Boolean),
        ('defect_type', String),
        ('inspection_actions', String)])

## 중복 제거 (전처리 코드로 돌리게 된다면 삭제)

In [7]:
# dedup_cols = [
#     'report_number',
#     'date_of_event', 
#     'manufacturer',
#     'udi_di',
#     'device_0_lot_number',
#     'device_0_udi_public'
# ]

# # Unknown / N/A 패턴 리스트
# na_patterns = r'^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$'

In [8]:
# # 일단 중복된 행들만 확인
# # 조합 (6개 컬럼)의 개수
# lf_with_cnt = lf.with_columns(
#     pl.len().over(dedup_cols).alias('duplicate_cnt')
# )

# # cnt가 2 이상이 경우에만 
# # cnt가 1인 경우에는 삭제할 필요가 없으므로
# lf_duplicates_only = lf_with_cnt.filter(
#     pl.col('duplicate_cnt') >= 2
# )

# duplicate_cnt = lf_duplicates_only.select(pl.len()).item()
# print(f"중복된 행의 개수: {duplicate_cnt:,}개")

# unique_cnt = lf.unique(subset=dedup_cols, maintain_order=True).select(pl.len()).item()
# print(f"유일한 행의 개수: {unique_cnt:,}개")

In [9]:
# def remove_na_values(lf, dedup_cols, na_patterns, verbose=True):
#     """
#     Na / Unknwon 값이 있는 행을 제거하는 함수
    
#     작동방식 :
#     1. 각 컬럼에 대해 유효한 값인지 체크
#     2. 모든 조건은 포함한(모두 만족하는) 행으로 필터링
#     3. 필터 적용

#     Parameters:
#     lf : polars.DataFrame
#         원본 DF
#     dedup_cols : list
#         -> 모두 유효해야 함
#     na_patterns : str
#         NA / Unknown 패턴 정규식
#     verbose : bool
#         -> 현재 진행상황 출력해서 확인할 수 있게

#     Returns: 
#     polars.LazyFrame
#         비어진 값이나 모르는 값이 없는 DF
#     """

#     # 진행상확 확인
#     if verbose:
#         print("na 값 제거")
#         print(f"패턴: {na_patterns}")

#     # 제거 되기 전 개수 확인
#     before_cnt = lf.select(pl.len()).item()
#     if verbose:
#         print(f"제거 전 행 개수 : {before_cnt:,}개")
    
#     # ===
#     # 각 컬럼별로 필터 조건
#     # ===
#     conditions = []

#     for col in dedup_cols:
#         # 컬럼이 존재하는지 확인
#         if col in lf.collect_schema().names():
#             # 유효한 값의 조건
#             # null / na_patterns 패턴에 매칭되지 않는 값
#             cond = (
#                 pl.col(col).is_not_null()
#                 &
#                 ~ pl.col(col).cast(pl.Utf8).str.to_uppercase().str.contains(na_patterns)
#                 # ~ 은 not 연산자
#             )
#             conditions.append(cond)

#             if verbose:
#                 print(f"컬럼 '{col}'에 대해 na/unknown 값 제거 조건 추가")
#         else:
#             if verbose:
#                 print(f"컬럼 '{col}'이(가) 존재하지 않음. 건너뜀")
        
# # 예외처리
#     if not conditions:
#         if verbose:
#             print("제거할 조건이 없음. 원본 DF 반환")
#         return lf

# # ===
# # 모든 조건을 and 조건으로 결합
# # (모든 조건을 만족해야 함)
# # ===
#     final_condition = conditions[0]
#     for cond in conditions[1:]:
#         final_condition = final_condition & cond

# # 필터 적용
#     lf_cleaned = lf.filter(final_condition)

# # 결과
#     after_cnt = lf_cleaned.select(pl.len()).item()
#     removed_cnt = before_cnt - after_cnt

#     if verbose:
#         print(f"제거 후 행 개수 : {after_cnt:,}개")
#         print(f"제거된 행 개수 : {removed_cnt:,}개")

#         return lf_cleaned

In [10]:
# def analyze_duplicates(lf, group_cols, verbose = True):
#     """
#     중복 데이터 분석 함수들

#     작동 방식 
#     1. 전체 개수 확인
#     2. 고유(unique) 개수 확인
#     3. 전체 - 고유 = 중복 개수 확인

#     Parameters:
#     lf : polars.DataFrame -> 원본
#     dedup_cols : list -> 중복 확인 컬럼
#     verbose : bool -> 진행상황 출력 여부

#     returns:

#     tuple : (전체 개수, 고유 개수, 중복 개수)
#     """

#     if verbose:
#         print(f" 중복 확인")

#     # 전체 개수
#     total_cnt = lf.select(pl.len()).item()

#     # 고유 개수
#     unique_cnt = lf.unique(
#         subset = group_cols, # 중복 ㅎ판단
#         maintain_order = True
#     ).select(pl.len()).item()

#     # 중복 개수
#     duplicate_cnt = total_cnt - unique_cnt

#     if verbose:
#         print(f"전체 개수 : {total_cnt:,}개")
#         print(f"고유 개수 : {unique_cnt:,}개")
#         print(f"중복 개수 : {duplicate_cnt:,}개")
#         for i, col in enumerate(dedup_cols, start=1):
#             print(f"{i}. {col}")

#     return total_cnt, unique_cnt, duplicate_cnt

In [11]:
# def remove_duplicates(lf, dedup_cols, keep = 'first', verbose = True):
#     """
#     중복 데이터 제거 함수

#     작동 방식
#     1. dedup_cols 기준으로 중복 판단
#     2. keep 옵션에 따라 첫번째/마지막 행 유지
#     3. 중복 제거된 DF 반환

#     Parameters:
#     lf : polars.DataFrame -> 원본 DF
#     dedup_cols : list -> 중복 판단 컬럼 리스트
#     keep : str -> 'first' or 'last'
#         'first' : 첫번째 행 유지
#         'last' : 마지막 행 유지
#     verbose : bool -> 진행상황 출력 여부

#     Returns:
#     polars.DataFrame -> 중복 제거된 DF
#     """

#     if verbose:
#         print("중복 제거 시작")
#         print(f"중복 판단 컬럼: {dedup_cols}")
#         print(f"유지 옵션: {keep}")

#     # 중복 제거
#     lf_deduped = lf.unique(
#         subset = dedup_cols,
#         maintain_order = True,
#         keep = 'first'
#     )

#     if verbose:
#         before_cnt = lf.select(pl.len()).item()
#         after_cnt = lf_deduped.select(pl.len()).item()
#         removed_cnt = before_cnt - after_cnt

#         print(f"제거 전 행 개수 : {before_cnt:,}개")
#         print(f"제거 후 행 개수 : {after_cnt:,}개")
#         print(f"제거된 행 개수 : {removed_cnt:,}개")

#     return lf_deduped

In [12]:
# # 완전한 파이프라인
# lf_all_cleaned = remove_na_values(lf, dedup_cols, na_patterns)
# total, unique, dup = analyze_duplicates(lf_all_cleaned, dedup_cols)
# lf_all_final = remove_duplicates(lf_all_cleaned, dedup_cols, keep='first')


## 클래스3에서 진행되는거겠죠? (전처리 된 파일의 코드를 뽑은 프롬프트로 실행하면 삭제)

In [13]:
# print("="*80)
# print("데이터 타입 통일 및 Class 3 필터링")
# print("="*80)

# # 1. device_class 컬럼 찾기
# device_class_cols = [
#     col for col in lf.collect_schema().names()
#     if 'device_' in col and 'openfda_device_class' in col
# ]

# print(f"\n찾은 컬럼: {device_class_cols}")

# # 3. String으로 통일
# print(f"\nString 타입으로 통일 중...")
# lf = lf.with_columns([
#     pl.col(col).cast(pl.Utf8).alias(col) for col in device_class_cols
# ])

# print(f"\n변환 후 타입:")
# for col in device_class_cols:
#     dtype = lf.select(pl.col(col)).collect_schema()[col]
#     print(f"  {col}: {dtype}")

In [14]:
# lf_class3 = lf.filter()
# row_count = lf_class3.select(pl.len()).collect().item()
# print(f"{row_count:,}개")

In [15]:
# lf
# = lf_class3.clone()

In [16]:
# lf_class3 = lf.clone()  # 필요시 사용

# 제품코드별

## 1. 고장 유형 탐색 (EDA)

###  1-1. 이상 사례 발생 top 10 제품코드 / 결함 유형 발생 비율

In [17]:
# 함수화 
def get_top10_products(lf, product_col = 'product_code'):
    """
    코드 중에서 이상 사례 발생 수가 높은 상위 10개의 제품코드 (class 3)
    
    Args : 
        lf : DF
        product_col = 제품코드 컬럼명

    Returns : 
        top 10 결과 , 제품 코드 리스트 
    """
    print("=" * 50)
    print("top 10  제품코드 분석")
    print("=" * 50)

   # 전체 제품 코드에서 이상 사례 발생 수가 높은 상위 10개 제품 코드 확인
    top10_result = (
        lf
        .group_by(product_col)
        .agg(pl.len().alias("total_count"))
        .sort("total_count", descending=True)
        .head(10)
        .collect()
    )
    
    print("top 10 제품코드")
    print(top10_result)

    # 상위 10개 제품 코드 리스트화
    top10_product_codes = top10_result[product_col].to_list()
        # top10_result[product_col].to_list()

    return top10_result, top10_product_codes

In [18]:
top10_result, top10_product_codes = get_top10_products(lf)

top 10  제품코드 분석
top 10 제품코드
shape: (10, 2)
┌──────────────┬─────────────┐
│ product_code ┆ total_count │
│ ---          ┆ ---         │
│ str          ┆ u32         │
╞══════════════╪═════════════╡
│ LGW          ┆ 2135        │
│ DTB          ┆ 1601        │
│ LWS          ┆ 1233        │
│ NVN          ┆ 1144        │
│ FTR          ┆ 886         │
│ OZD          ┆ 649         │
│ NIK          ┆ 647         │
│ DSQ          ┆ 615         │
│ LWP          ┆ 611         │
│ MGB          ┆ 479         │
└──────────────┴─────────────┘


In [19]:
def get_total_ratio(lf):
    """
    전체 기기에서의 각 결함 유형 발생 비율

    Args :
        lf : 분석할 DF
        defect_col : 분석할 컬럼 (결함 유형)

    Returns : 
    """
    print("=" * 50)
    print("기기 결함 유형")
    print("=" * 50)

    # 전체 기기 이상 사례 데이터에서 각 결함 유형의 발생 비율 확인
    ratio_result = (
        lf
        .group_by("defect_type")  # 결함 유형별로 그룹화
        .agg(pl.len().alias("count"))
        .with_columns(
            (pl.col("count") / pl.col("count").sum() * 100)
            .round(2)
            .alias("percentage")
        )
        .sort("percentage", descending=True)
        .collect()  # LazyFrame이면 필요
    )

    print(ratio_result.to_pandas())

    return ratio_result

In [20]:
ratio_result = get_total_ratio(lf)

기기 결함 유형
                    defect_type  count  percentage
0                         Other   2465       24.65
1               Sensor/Accuracy   2103       21.03
2                       Unknown   1599       15.99
3         Mechanical/Structural   1404       14.04
4              Electrical/Power   1050       10.50
5            Functional Failure    935        9.35
6    Communication/Connectivity    236        2.36
7             User/Human Factor     80        0.80
8                          None     32        0.32
9                   Alarm/Alert     31        0.31
10           Software/Interface     26        0.26
11      Sterility/Contamination     19        0.19
12  Environmental/Compatibility     17        0.17
13           Labeling/Packaging      3        0.03


### 1-2. 어떤 제품코드가 어떤 고장 유형이 많이 발생하는가?

In [21]:
def analyze_product_defects(lf, top10_product_codes,
                             product_col = 'product_code'):
    """
    각 제품 코드별 어떤 결함이 많이 발생했는지 확인

    Args : 
        lf : 분석할 DF
        top10_product_codes : 제품코드 리스트 
        product_col : 제품코트 컬럼
        defect_col : 고장 유형 컬럼

    Returns : 
    """

    print("=" * 50)
    print("제품 코드별 결함유형")
    print("=" * 50)
    
    # product_code 별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
    product_code_result = (
        lf
        .filter(pl.col(product_col).is_in(top10_product_codes))
        .group_by([product_col, "defect_type"])
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .group_by(product_col)
        .head(3)  # 각 그룹별 상위 3개
        .with_columns(
            (pl.col("count") / pl.col("count").sum().over(product_col) * 100)
            .round(2)
            .alias("percentage")
        )
        .sort([product_col, "percentage"], descending=[False, True])
        .collect()
        .to_pandas()
    )

    print(product_code_result)

    return product_code_result

In [22]:
product_code_result = analyze_product_defects(
    lf, top10_product_codes
)

제품 코드별 결함유형
   product_code            defect_type  count  percentage
0           DSQ                Unknown    204       45.84
1           DSQ                  Other    126       28.31
2           DSQ       Electrical/Power    115       25.84
3           DTB        Sensor/Accuracy    840       63.25
4           DTB                  Other    311       23.42
5           DTB                Unknown    177       13.33
6           FTR                  Other    377       43.53
7           FTR  Mechanical/Structural    356       41.11
8           FTR                Unknown    133       15.36
9           LGW       Electrical/Power    493       40.88
10          LGW                Unknown    390       32.34
11          LGW  Mechanical/Structural    323       26.78
12          LWP                  Other    193       44.88
13          LWP       Electrical/Power    124       28.84
14          LWP        Sensor/Accuracy    113       26.28
15          LWS        Sensor/Accuracy    442       50.75
16

## 통계

### 1) 카이제곱검정
- **목적**: 특정 제품코드에서 특정 결함이 과대표되는지 검증
- **귀무가설**: 결함 유형과 제품코드는 독립적이다
- **적용**:
    - 관측 빈도 vs 기대 빈도
    - 제품코드 A에서 '배터리 결함' 발생 빈도가 전체 평균 대비 유의하게 높은가?
- **결과 해석**: p < 0.05이면 특정 패턴 존재

### 2) 검정 이후 표준화 잔차 확인

- **목적**: 어떤 셀이 기대치를 크게 벗어나는지 정량화
- **공식**: `(관측값 - 기대값) / sqrt(기대값)`
- **기준**: |z| > 2이면 주목, |z| > 3이면 매우 이례적

### 3) 효과 크기 - Cramer’s V

- **목적**: 통계적 유의성과 별개로 실질적 연관 강도 측정
- **범위**: 0(독립) ~ 1(완전 연관)

In [23]:
def prepare_chi2_data(lf, top10_product_codes,
                        product_col ='product_code'):
    """
    카이제곱 검정
    - top10 제품만 필터링
    - Unknown, Other 제거
    """

    print("=" * 50)
    print("카이제곱 검정용 데이터")
    print("=" * 50)

    # 상위 10개 제품만 필터링 + Pandas로 변환
    lf_filtered = (
        lf
        .filter(
            (pl.col(product_col).is_in(top10_product_codes))
            & (~pl.col("defect_type").is_in(["Unknown", "Other"])))
        .select([product_col, "defect_type"])
        .collect()
        .to_pandas()  # Pandas로 변환
        )

    print(f"필터링 완료: {len(lf_filtered): }행")
    print(f"제품 종류: {lf_filtered[product_col].nunique()}개")
    print(f"결함 종류: {lf_filtered['defect_type'].nunique()}개")

    return lf_filtered

In [24]:
lf_filtered = prepare_chi2_data(
    lf,
    top10_product_codes
)

카이제곱 검정용 데이터
필터링 완료:  5904행
제품 종류: 10개
결함 종류: 11개


In [25]:
chi2_tester = Chi2Test()
chi2_result = chi2_tester.execute(lf_filtered, 'product_code', 'defect_type')


[기대빈도 확인]
----------------------------------------
최소 기대빈도: 0.03
5 미만 셀: 52/110 (47.3%)
⚠️ 주의: Fisher's exact test 사용 권장
   (기대빈도가 너무 작아 카이제곱 검정 부정확)

[기대빈도]
defect_type   Alarm/Alert  Communication/Connectivity  Electrical/Power  \
product_code                                                              
DSQ                  1.50                       11.39             50.69   
DTB                  5.83                       44.37            197.41   
FTR                  1.94                       14.75             65.62   
LGW                  7.63                       58.12            258.59   
LWP                  1.84                       13.99             62.25   
LWS                  4.62                       35.18            156.50   
MGB                  0.28                        2.16              9.60   
NIK                  2.26                       17.23             76.65   
NVN                  3.57                       27.18            120.93   
OZD             

defect_type,Alarm/Alert,Communication/Connectivity,Electrical/Power,Environmental/Compatibility,Functional Failure,Labeling/Packaging,Mechanical/Structural,Sensor/Accuracy,Software/Interface,Sterility/Contamination,User/Human Factor
product_code,,,,,,,,,,,
DSQ,1.23,1.07,9.03,5.72,4.15,-0.38,-0.22,-9.88,0.66,-0.96,-0.44
DTB,-2.00,-6.66,-11.99,-1.79,-7.22,-0.75,-6.83,22.36,-2.21,0.76,-3.36
FTR,-1.39,-3.84,-8.10,-1.03,-6.60,4.19,28.64,-11.46,-1.27,0.75,-1.79
LGW,-2.76,15.46,14.58,-0.58,-0.15,-0.86,-1.22,-14.41,0.24,0.15,2.09
LWP,0.12,-3.47,7.83,-1.00,1.69,-0.42,-4.96,-1.05,1.98,-1.06,-2.18
LWS,7.62,-4.92,3.88,0.29,-2.49,-0.67,-8.24,7.26,-1.97,-0.49,-3.16
MGB,-0.53,-1.47,-3.10,-0.39,8.70,-0.17,-0.51,-4.39,-0.49,-0.42,9.67
NIK,-0.17,3.32,-3.50,2.48,2.63,-0.47,-5.68,4.48,0.07,-1.18,-1.59
NVN,-1.36,-5.21,-10.63,-1.40,2.44,-0.59,6.31,3.39,-1.73,1.90,-3.04


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   product_code과(와) defect_type은(는) 관련이 있음   효과 크기: 약한 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • DSQ - Electrical/Power: 예상보다 많음 (잔차=9.03)
   • DSQ - Environmental/Compatibility: 예상보다 많음 (잔차=5.72)
   • DSQ - Functional Failure: 예상보다 많음 (잔차=4.15)
   • DSQ - Sensor/Accuracy: 예상보다 적음 (잔차=-9.88)
   • DTB - Communication/Connectivity: 예상보다 적음 (잔차=-6.66)
   • DTB - Electrical/Power: 예상보다 적음 (잔차=-11.99)
   • DTB - Functional Failure: 예상보다 적음 (잔차=-7.22)
   • DTB - Mechanical/Structural: 예상보다 적음 (잔차=-6.83)
   • DTB - Sensor/Accuracy: 예상보다 많음 (잔차=22.36)
   • DTB - Software/Interface: 예상보다 적음 (잔차=-2.21)
   • DTB - User/Human Factor: 예상보다 적음 (잔차=-3.36)
   • FTR - Communication/Connectivity: 예상보다 적음 (잔차=-3.84)
   • FTR - Electrical/Power: 예상보다 적음 (잔차=-8.10)
   • FTR - Functional Failure: 예상보다 적음 (잔차=-6.60)
   • FTR - Labeling/Packaging: 예상보다 많음 (잔차=4.19)
   • FTR - Mechanical/Structural: 예상보다 많음

### 오즈비
- **목적**: 특정 제품에서 특정 결함 발생 확률 비교
- **공식**: `OR = (제품A 결함발생 오즈) / (다른 제품 결함발생 오즈)`
- **결과 예시**: "인슐린 펌프 X의 배터리 결함 OR = 4.2 (95% CI: 3.1-5.7)"
    - 해석: 다른 제품 대비 4.2배 높은 확률
    - 이후 다시 **카이제곱 검정**으로 특정 제품 대비 전체 제품 오즈비 비교가 유의한지 확인도 가능

In [26]:
import numpy as np
import pandas as pd

def calculate_odds_ratio(
    contingency_table: pd.DataFrame,
    target_product: str,
    target_defect: str,
    chi2_p_value: float = None,
    alpha: float = 0.05
) -> dict:
    """
    분할표에서 특정 제품-결함 조합의 오즈비 계산
    
    Parameters
    ----------
    contingency_table : pd.DataFrame
        pd.crosstab으로 생성한 분할표
        index: 제품 코드, columns: 결함 유형
    target_product : str
        분석 대상 제품 코드
    target_defect : str
        분석 대상 결함 유형
    chi2_p_value : float, optional
        Chi2Test()의 p-value (제공 시 사용)
    alpha : float
        유의수준 (default: 0.05)
    
    Returns
    -------
    dict
        - odds_ratio: 오즈비
        - ci_lower: 95% 신뢰구간 하한
        - ci_upper: 95% 신뢰구간 상한
        - p_value: 카이제곱 검정 p-value
        - interpretation: 해석
        - a, b, c, d: 2x2 분할표 셀 값
    
    Example
    -------
    >>> contingency_table = pd.crosstab(lf['product_code'], lf['defect_type'])
    >>> chi2_result = chi2_tester.execute(lf, 'product_code', 'defect_type')
    >>> result = calculate_odds_ratio(contingency_table, 'INS', '배터리결함', chi2_result.p_value)
    >>> print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")
    OR = 4.17 (95% CI: 2.52-6.89)
    
    Notes
    -----
    자동으로 2x2 분할표 생성:
                    타겟 결함    기타 결함
    타겟 제품          a           b
    기타 제품          c           d
    """
    
    # a: 타겟 제품 & 타겟 결함
    a = contingency_table.loc[target_product, target_defect] if target_product in contingency_table.index and target_defect in contingency_table.columns else 0
    
    # b: 타겟 제품 & 기타 결함
    b = contingency_table.loc[target_product, :].sum() - a if target_product in contingency_table.index else 0
    
    # c: 기타 제품 & 타겟 결함
    c = contingency_table.loc[:, target_defect].sum() - a if target_defect in contingency_table.columns else 0
    
    # d: 기타 제품 & 기타 결함
    total = contingency_table.values.sum()
    d = total - a - b - c
    
    print(f"\n[2x2 분할표: {target_product} - {target_defect}]")
    print(pd.DataFrame(
        [[a, b], [c, d]],
        index=[f'{target_product}', '기타 제품'],
        columns=[f'{target_defect}', '기타 결함']
    ))
    print()
    
    # 0 처리 (연속성 수정)
    if b == 0 or c == 0 or a == 0 or d == 0:
        a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5
    
    # 오즈비 계산
    odds_ratio = (a * d) / (b * c)
    
    # 로그 오즈비의 표준오차 (SE)
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    
    # 95% 신뢰구간 (로그 스케일에서 계산 후 역변환)
    z_critical = 1.96  # 95% CI
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - z_critical * se_log_or)
    ci_upper = np.exp(log_or + z_critical * se_log_or)
    
    # p-value (Chi2Test 결과 사용)
    p_value = chi2_p_value
    
    # 해석
    if p_value is not None and p_value < alpha:
        if odds_ratio > 1:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {odds_ratio:.2f}배 높음 (p={p_value:.4f}, 유의함)"
        else:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {1/odds_ratio:.2f}배 낮음 (p={p_value:.4f}, 유의함)"
    elif p_value is not None:
        interpretation = f"오즈비가 유의하지 않음 (p={p_value:.4f})"
    else:
        interpretation = "p-value 미제공"
    
    return {
        'odds_ratio': odds_ratio,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'interpretation': interpretation,
        'a': int(a) if a == int(a) else a,
        'b': int(b) if b == int(b) else b,
        'c': int(c) if c == int(c) else c,
        'd': int(d) if d == int(d) else d
    }

# # 사용 예시
# if __name__ == "__main__":
#     # 예시 분할표 생성
#     data = {
#         '배터리결함': [50, 30, 20],
#         '화상': [100, 80, 60],
#         '오작동': [50, 40, 30]
#     }
#     contingency_table = pd.DataFrame(data, index=['INS', 'PUM', 'DEF'])
    
#     print("전체 분할표:")
#     print(contingency_table)
    
#     # 오즈비 계산
#     result = calculate_odds_ratio(
#         contingency_table=contingency_table,
#         target_product='INS',
#         target_defect='배터리결함',
#         chi2_p_value=0.0001
#     )
    
#     print(f"OR = {result['odds_ratio']:.2f}")
#     print(f"95% CI: ({result['ci_lower']:.2f}, {result['ci_upper']:.2f})")
#     print(f"해석: {result['interpretation']}")

In [27]:
# 1. 분할표 생성
contingency_table = pd.crosstab(
    lf_filtered['product_code'], 
    lf_filtered['defect_type']
)

# 2. 카이제곱 검정 생략: chi2_result 이미 계산됨

# 3. 오즈비 계산 (분할표 + 타겟만 지정)
result = calculate_odds_ratio(
    contingency_table=contingency_table,
    target_product='OZP',      # 분석할 제품 코드
    target_defect='Sensor/Accuracy',  # 분석할 결함 유형
    chi2_p_value=chi2_result.p_value
)

print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")


[2x2 분할표: OZP - Sensor/Accuracy]
       Sensor/Accuracy  기타 결함
OZP                  0      0
기타 제품             2103   3801

OR = 1.81 (95% CI: 0.04-91.12)


# 제조사 - 모델 별

## EDA

###  어느 제조사에서 어떤 고장 유형이 많이 발생하는가?

In [28]:
def analyze_manufacturer_defacts(lf, manufacturer_col ='manufacturer'):
    """
    제조업체 별 결함이 얼마나 발생하는지 분석

    Args : 
        manufacturer_col : 제조사 컬럼
    """
    print("=" * 50)
    print("제조사별 결함유형")
    print("=" * 50)

    # manufacturer별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
    manufacturer_result = (
        lf
        .filter(pl.col(manufacturer_col).is_not_null())
        .group_by([manufacturer_col, "defect_type"])
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        # .group_by(manufacturer_col)
        # .head(5)  # 각 그룹별 상위 5개
        .with_columns(
            (pl.col("count") / pl.col("count").sum().over(manufacturer_col) * 100)
            .round(2)
            .alias("percentage")
        )
        .sort([manufacturer_col, "percentage"], descending=[False, True])
        .collect()
    )

    print(manufacturer_result)

    return manufacturer_result

In [29]:
manufacturer_result = analyze_manufacturer_defacts(lf)


제조사별 결함유형
shape: (145, 4)
┌─────────────────┬────────────────────┬───────┬────────────┐
│ manufacturer    ┆ defect_type        ┆ count ┆ percentage │
│ ---             ┆ ---                ┆ ---   ┆ ---        │
│ str             ┆ str                ┆ u32   ┆ f64        │
╞═════════════════╪════════════════════╪═══════╪════════════╡
│ ABBOTT VASCULAR ┆ Other              ┆ 350   ┆ 89.06      │
│ ABBOTT VASCULAR ┆ Unknown            ┆ 31    ┆ 7.89       │
│ ABBOTT VASCULAR ┆ Functional Failure ┆ 6     ┆ 1.53       │
│ ABBOTT VASCULAR ┆ User/Human Factor  ┆ 6     ┆ 1.53       │
│ ABIOMED         ┆ Other              ┆ 182   ┆ 28.04      │
│ …               ┆ …                  ┆ …     ┆ …          │
│ THORATEC        ┆ User/Human Factor  ┆ 3     ┆ 0.58       │
│ THORATEC        ┆ Alarm/Alert        ┆ 3     ┆ 0.58       │
│ THORATEC        ┆ Sensor/Accuracy    ┆ 2     ┆ 0.39       │
│ THORATEC        ┆ Software/Interface ┆ 1     ┆ 0.19       │
│ VASORUM         ┆ Unknown            ┆ 1  

In [30]:
manufacturer_result.filter((pl.col("count") >= 3)
                           & (~pl.col("defect_type")
                              .is_in(["Unknown", "Other"]))).unique("manufacturer")

manufacturer,defect_type,count,percentage
str,str,u32,f64
"""ABBOTT VASCULAR""","""Functional Failure""",6,1.53
"""ABIOMED""","""Functional Failure""",111,17.1
"""ADVANCED NEUROMODULATION SYSTE…","""Sensor/Accuracy""",6,28.57
"""ALLERGAN""","""Mechanical/Structural""",61,32.8
"""BIOTRONIK SE & KG""","""Electrical/Power""",24,18.9
…,…,…,…
"""SALUDA MEDICAL""","""Mechanical/Structural""",7,43.75
"""SIENTRA""","""Mechanical/Structural""",7,18.92
"""ST JUDE MEDICAL""","""Sensor/Accuracy""",435,21.88


### 제조사-모델 조합 Top N

In [31]:
lf.select(
    pl.len().alias("total"),
    pl.col("udi_di").null_count().alias("udi_null")
).with_columns(
    (pl.col("udi_null") / pl.col("total") * 100).alias("null_pet")
).collect()

total,udi_null,null_pet
u32,u32,f64
10000,0,0.0


In [32]:
def analyze_manufacturer_model_top_n(
    lf,
    top_n: int = 10,
    manufacturer_col: str = "manufacturer",
    model_col: str = "udi_di",
    verbose: bool = True,
    name_width: int = 65,
) -> pl.DataFrame:
    """
    제조사-모델(또는 브랜드) 조합별 이상 보고 수 Top N 분석

    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_n : int, default=10
        상위 N개 조합 표시
    manufacturer_col : str, default="manufacturer"
        제조사 컬럼명
    model_col : str, default="udi_di"
        모델/브랜드 이름 컬럼명
    verbose : bool, default=True
        출력 여부 (True면 보기 좋은 표 형태로 출력)
    name_width : int, default=65
        verbose 출력 시 "제조사-모델" 컬럼 표시 폭(문자 수)

    Returns
    -------
    polars.DataFrame
        제조사-모델 조합별 집계 결과
        - manufacturer_model : str (제조사 - 모델/브랜드 조합 문자열)
        - report_count       : int (해당 조합의 보고 건수)
        - percentage         : float (전체 대비 비율, %)

    Examples
    --------
    >>> # 상위 20개 제조사-모델 조합
    >>> top_models = analyze_manufacturer_model_top_n(lf, top_n=20)

    >>> # 상위 10개만
    >>> top_10 = analyze_manufacturer_model_top_n(lf, top_n=10)

    >>> # 출력 폭 늘리기
    >>> top_10 = analyze_manufacturer_model_top_n(lf, top_n=10, name_width=80)
    """
    # lf = lf.lazy() if isinstance(lf, pl.DataFrame) else lf

    # 컬럼 존재 체크
    schema = lf.collect_schema().names()
    missing = [c for c in (manufacturer_col, model_col) if c not in schema]
    if missing:
        raise ValueError(
            f"컬럼 없음: {missing}\n"
            f"비슷한 후보: {[c for c in schema if 'brand' in c.lower() or 'model' in c.lower() or 'manufacturer' in c.lower()][:30]}"
        )

    # 전체 개수
    total = lf.select(pl.len()).collect().item()

    # 제조사-모델 조합 생성 및 집계
    result = (
        lf.with_columns(
            pl.when(pl.col(manufacturer_col).is_not_null() & pl.col(model_col).is_not_null())
              .then(
                  pl.col(manufacturer_col).cast(pl.Utf8)
                  + pl.lit(" - ")
                  + pl.col(model_col).cast(pl.Utf8)
              )
              .when(pl.col(manufacturer_col).is_not_null())
              .then(
                  pl.col(manufacturer_col).cast(pl.Utf8)
                  + pl.lit(" - (모델 정보 없음)")
              )
              .otherwise(pl.lit("(정보 없음)"))
              .alias("manufacturer_model")
        )
        .group_by("manufacturer_model")
        .agg(pl.len().alias("report_count"))
        .with_columns((pl.col("report_count") / total * 100).round(2).alias("percentage"))
        .sort(pl.col("report_count"), descending=True)
        .head(top_n)
        .collect()
    )

    # 보기 좋은 출력
    if verbose:
        print("=" * 90)
        print(f"제조사-모델 조합별 이상 보고 수 Top {top_n} (전체: {total:,}건)")
        print("=" * 90)
        print(f"{'순위':>3}  {'제조사-모델':<{name_width}} {'건수':>10} {'비율':>9}")
        print("-" * 90)

        for i, row in enumerate(result.iter_rows(named=True), 1):
            name = (row.get("manufacturer_model") or "").replace("\n", " ")
            name_disp = name[:name_width]
            print(
                f"{i:>3}  {name_disp:<{name_width}} "
                f"{int(row['report_count']):>10,} {row['percentage']:>8.2f}%"
            )

        print("=" * 90)

    return result


In [33]:
top_10_models = analyze_manufacturer_model_top_n(lf, top_n=10)

제조사-모델 조합별 이상 보고 수 Top 10 (전체: 10,000건)
 순위  제조사-모델                                                                    건수        비율
------------------------------------------------------------------------------------------
  1  MEDTRONIC - 00643169633759                                               406     4.06%
  2  ST JUDE MEDICAL - 05415067017246                                         372     3.72%
  3  ABBOTT VASCULAR - 08717648235184                                         336     3.36%
  4  ABIOMED - 00813502011876                                                 309     3.09%
  5  ST JUDE MEDICAL - 05415067031419                                         260     2.60%
  6  THORATEC - 00813024013297                                                256     2.56%
  7  MEDTRONIC - 00643169633742                                               217     2.17%
  8  MEDTRONIC - 00643169356566                                               196     1.96%
  9  MEDTRONIC - 00643169781702          

### 특정 제조사-모델의 상세 분석

In [34]:
def analyze_specific_combo_issues(
    lf,
    manufacturer_model: str,
    manufacturer_col: str = "manufacturer",
    model_col: str = "udi_di",
):
    """
    특정 제조사-모델(조합 문자열) 1개에 대해 상세 분포를 출력/반환하는 함수.

    이 함수는 Top N 집계 함수에서 사용한 로직과 "완전히 동일한 방식"으로
    manufacturer_model(= 제조사 + " - " + 모델/브랜드) 조합 컬럼을 다시 생성한 뒤,
    입력받은 manufacturer_model 문자열과 정확히 일치하는 행만 필터링하여 분석한다.

    주로 다음 상황에서 유용하다.
    - Top N 결과에서 특정 조합(예: 1위/3위)을 클릭하듯 “그 조합 그대로” 상세 분석하고 싶을 때
    - 제조사/모델을 각각 비교(==)했을 때 공백/특수문자/잘린 출력 등의 이유로 매칭이 실패하는 문제를 피하고 싶을 때

    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    manufacturer_model : str
        분석 대상 조합 문자열.
        예: "MEDTRONIC ... - PUMP 1886 780G ...".
        권장: Top N 결과에서 `top_n_lf.select("manufacturer_model").item(row_idx, 0)`로
        "원문 그대로" 가져와 넣는 방식(복붙보다 안전).
    manufacturer_col : str, default="manufacturer"
        제조사 컬럼명
    model_col : str, default="udi_di"
        모델/브랜드 컬럼명

    Returns
    -------
    dict | None
        데이터가 있으면 dict를 반환한다.
        - total : int
            필터링된 보고 건수
        - event_dist : polars.DataFrame
            event_type별 건수/비율 분포 (count, percentage 포함)

        해당 조합에 대한 데이터가 없으면 None을 반환한다.

    Examples
    --------
    >>> # 1) Top N 집계 후, 1위 조합 상세 분석 (복붙 없이 안전)
    >>> top_10 = analyze_manufacturer_model_top_n(lf, top_n=10)
    >>> top1_combo = top_10.select("manufacturer_model").item(0, 0)
    >>> analyze_specific_combo_issues(lf, top1_combo)

    >>> # 2) 3위 조합 상세 분석
    >>> top3_combo = top_10.select("manufacturer_model").item(2, 0)  # 0-based index
    >>> analyze_specific_combo_issues(lf, top3_combo)

    >>> # 3) 문자열을 직접 넣어서 분석 (오타/공백 차이 있으면 0건이 될 수 있음)
    >>> analyze_specific_combo_issues(
    ...     lf,
    ...     "MEDTRONIC PUERTO RICO OPERATIONS CO. - PUMP 1886 780G OUS BLE PUMP MG/DL"
    ... )
    """
    # lf = lf.lazy() if isinstance(lf, pl.DataFrame) else lf

    # TopN 만들 때랑 동일하게 조합 컬럼 생성
    combo_expr = (
        pl.when(pl.col(manufacturer_col).is_not_null() & pl.col(model_col).is_not_null())
          .then(
              pl.col(manufacturer_col).cast(pl.Utf8)
              + pl.lit(" - ")
              + pl.col(model_col).cast(pl.Utf8)
          )
          .when(pl.col(manufacturer_col).is_not_null())
          .then(
              pl.col(manufacturer_col).cast(pl.Utf8)
              + pl.lit(" - (모델 정보 없음)")
          )
          .otherwise(pl.lit("(정보 없음)"))
          .alias("manufacturer_model")
    )

    filtered = (
        lf.with_columns(combo_expr)
          .filter(pl.col("manufacturer_model") == pl.lit(manufacturer_model))
    )

    total = filtered.select(pl.len()).collect().item()
    if total == 0:
        print(f"⚠️  '{manufacturer_model}'에 대한 데이터가 없습니다.")
        return None

    print("=" * 100)
    print(f"제조사-모델 상세 분석: {manufacturer_model}")
    print("=" * 100)
    print(f"총 보고 건수: {total:,}건\n")

    event_dist = (
        filtered.group_by("event_type")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / total * 100).round(2).alias("percentage"))
        .sort(pl.col("count"), descending=True)
        .collect()
    )

    print("1) 사건 유형 분포")
    for r in event_dist.iter_rows(named=True):
        e = r["event_type"] if r["event_type"] else "(NULL)"
        print(f"  {e:<20} {r['count']:>10,}건 ({r['percentage']:>5.2f}%)")

    return {"total": total, "event_dist": event_dist}


In [35]:
top1_combo = top_10_models.select("manufacturer_model").item(0, 0)
analyze_specific_combo_issues(lf, top1_combo)


제조사-모델 상세 분석: MEDTRONIC - 00643169633759
총 보고 건수: 406건

1) 사건 유형 분포
  Malfunction                 257건 (63.30%)
  Injury                      148건 (36.45%)
  Death                         1건 ( 0.25%)


{'total': 406,
 'event_dist': shape: (3, 3)
 ┌─────────────┬───────┬────────────┐
 │ event_type  ┆ count ┆ percentage │
 │ ---         ┆ ---   ┆ ---        │
 │ cat         ┆ u32   ┆ f64        │
 ╞═════════════╪═══════╪════════════╡
 │ Malfunction ┆ 257   ┆ 63.3       │
 │ Injury      ┆ 148   ┆ 36.45      │
 │ Death       ┆ 1     ┆ 0.25       │
 └─────────────┴───────┴────────────┘}

### Top N 제조사-모델의 Event Type 비교

In [36]:
def compare_top_models_event_types(
    lf,
    top_n: int = 5,
    manufacturer_col: str = "manufacturer",
    model_col: str = "udi_di",
):
    """
    상위 N개 제조사-모델(조합)별 사건 유형(event_type) 분포와 치명률(CFR)을 비교 출력한다.

    이 함수는 다음 순서로 동작한다.
    1) manufacturer_col + model_col을 결합해 `manufacturer_model`(조합 문자열) 컬럼을 생성
       - 제조사/모델이 모두 있으면: "제조사 - 모델"
       - 그 외(결측 등): "(정보 없음)"
    2) `manufacturer_model`별 보고 건수(total)를 집계하여 상위 top_n개 조합을 선정
    3) 선정된 각 조합에 대해
       - event_type 분포(건수/비율)를 계산
       - event_type이 "DEATH"인 건수를 기반으로 CFR(%) 계산

    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_n : int, default=5
        비교할 상위 제조사-모델 조합 수
    manufacturer_col : str, default="manufacturer"
        제조사 컬럼명
    model_col : str, default="device_0_model_number"
        모델 번호 컬럼명

    Returns
    -------
    None
        이 함수는 결과를 콘솔에 출력하는 용도이며, 별도의 DataFrame을 반환하지 않는다.
        (필요하면 반환형으로 확장 가능)

    Notes
    -----
    - 문자열 리터럴은 `pl.lit()`로 감싸 Lazy 최적화 과정에서 문자열이 "컬럼명"으로 오해되는 문제를 방지한다.
    - group_by 후 `.len()` 대신 `.agg(pl.len().alias("total"))`을 사용하여 컬럼명(`len`) 의존 문제를 피한다.
    - CFR 계산 시 event_type은 공백/대소문자 변형이 있을 수 있어, `strip + uppercase` 후 "DEATH"로 비교한다.

    Examples
    --------
    >>> # Top 5 제조사-모델 조합의 사건 유형 분포 및 CFR 비교
    >>> compare_top_models_event_types(lf, top_n=5)

    >>> # 모델 컬럼을 브랜드명으로 바꿔서 비교 (데이터에 해당 컬럼이 있을 때)
    >>> compare_top_models_event_types(
    ...     lf,
    ...     top_n=5,
    ...     manufacturer_col="manufacturer",
    ...     model_col="udi_di"
    ... )
    """
    # lf = lf.lazy() if isinstance(lf, pl.DataFrame) else lf

    # TopN 때와 동일한 조합 컬럼 생성 
    combo_expr = (
        pl.when(pl.col(manufacturer_col).is_not_null() & pl.col(model_col).is_not_null())
          .then(
              pl.col(manufacturer_col).cast(pl.Utf8)
              + pl.lit(" - ")
              + pl.col(model_col).cast(pl.Utf8)
          )
          .otherwise(pl.lit("(정보 없음)"))
          .alias("manufacturer_model")
    )

    # 상위 N개 제조사-모델 조합 선정
    top_models = (
        lf.with_columns(combo_expr)
          .group_by("manufacturer_model")
          .agg(pl.len().alias("total"))
          .sort(pl.col("total"), descending=True)
          .head(top_n)
          .collect()
    )

    print("=" * 120)
    print(f"Top {top_n} 제조사-모델별 사건 유형 분포 비교")
    print("=" * 120)

    # 루프에서 재사용(중복 with_columns 방지)
    lf_with_combo = lf.with_columns(combo_expr)

    for rank, row in enumerate(top_models.iter_rows(named=True), 1):
        mfr_model = row["manufacturer_model"]
        total = row["total"]

        # 해당 조합만 필터링
        model_data = lf_with_combo.filter(pl.col("manufacturer_model") == pl.lit(mfr_model))

        # Event Type 분포
        event_dist = (
            model_data.group_by("event_type")
            .agg(pl.len().alias("count"))
            .with_columns((pl.col("count") / total * 100).round(2).alias("percentage"))
            .sort(pl.col("count"), descending=True)
            .collect()
        )

        # CFR: DEATH 건수 / total
        death_count = (
            model_data.filter(
                pl.col("event_type").cast(pl.Utf8).str.strip_chars().str.to_uppercase() == "DEATH"
            )
            .select(pl.len())
            .collect()
            .item()
        )
        cfr = (death_count / total * 100) if total else 0

        print(f"\n[{rank}위] {mfr_model}")
        print(f"총 {total:,}건 | CFR: {cfr:.2f}%")
        print("-" * 120)
        print(f"{'사건 유형':<20} {'건수':>15} {'비율':>15}")
        print("-" * 120)

        for r in event_dist.iter_rows(named=True):
            event = r["event_type"] if r["event_type"] else "(NULL)"
            print(f"{event:<20} {r['count']:>15,} {r['percentage']:>14.2f}%")

    print("=" * 120)


In [37]:
# Top 3 모델의 Event Type 비교
compare_top_models_event_types(lf, top_n=5)

Top 5 제조사-모델별 사건 유형 분포 비교

[1위] MEDTRONIC - 00643169633759
총 406건 | CFR: 0.25%
------------------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
------------------------------------------------------------------------------------------------------------------------
Malfunction                      257          63.30%
Injury                           148          36.45%
Death                              1           0.25%

[2위] ST JUDE MEDICAL - 05415067017246
총 372건 | CFR: 0.00%
------------------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
------------------------------------------------------------------------------------------------------------------------
Injury                           371          99.73%
Malfunction                        1           0.27%

[3위] A

### 특정 제조사-모델 vs 파생 컬럼 종합 분석

In [38]:
def analyze_manufacturer_model_with_derived(lf, manufacturer, model_number = None,
                                           manufacturer_col='manufacturer',
                                           model_col='udi_di'):
    """
    제조사-모델의 파생 컬럼 포함 종합 분석
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임 (파생 컬럼 포함)
    manufacturer : str
        제조사명
    model_number : str, optional
        모델 번호 (None이면 해당 제조사 전체)
    manufacturer_col : str
        제조사 컬럼명
    model_col : str
        모델 번호 컬럼명
    
    Returns
    -------
    dict
        분석 결과
    
    Examples
    --------
    >>> # 특정 제조사-모델 분석
    >>> result = analyze_manufacturer_model_with_derived(lf, 'Medtronic', 'Model X123')
    
    >>> # 특정 제조사 전체
    >>> result = analyze_manufacturer_model_with_derived(lf, 'Medtronic')
    """
    # is_dataframe = isinstance(lf, pl.DataFrame)
    # if is_dataframe:
    #     lf = lf.lazy()
    
    # =============================================== 
    # udi-di가 존자하면 사용 
    # ===============================================
    # 필터링
    if model_number:
        filtered = lf.filter(
            (pl.col(manufacturer_col) == manufacturer) &
            (pl.col(model_col) == model_number)
        )
        title = f"{manufacturer} - {model_number}"
    else:
        filtered = lf.filter(pl.col(manufacturer_col) == manufacturer)
        title = f"{manufacturer} (전체 모델)"
    
    # # ================================================
    # #  udi-di가 존자하지 않을 경우 사용 (전처리 전 확인용)
    # # ================================================
    #     # "(모델 정보 없음)" 체크 추가
    # if model_number and "(모델 정보 없음)" in model_number:
    #     # null 체크로 변환
    #     filtered = lf.filter(
    #         (pl.col(manufacturer_col) == manufacturer) &
    #         (pl.col(model_col).is_null()) 
    #     )
    #     title = f"{manufacturer} - (모델 정보 없음)"
    # elif model_number:
    #     # 정상 모델 번호
    #     filtered = lf.filter(...)
    # else:
    #     # 제조사 전체
    #     filtered = lf.filter(...)


    total = filtered.select(pl.len()).collect().item()
    
    if total == 0:
        print(f"⚠️  '{title}'에 대한 데이터가 없습니다.")
        return None
    
    print("=" * 120)
    print(f"제조사-모델 종합 분석: {title}")
    print("=" * 120)
    print(f"\n◼️ 총 보고 건수: {total:,}건\n")
    
    # ==========================================
    # 1. 기본 통계
    # ==========================================
    print("1.  사건 유형 분포:")
    print("-" * 120)
    event_dist = filtered.group_by('event_type').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    for row in event_dist.iter_rows(named=True):
        event = row['event_type'] if row['event_type'] else "(NULL)"
        count = row['count']
        pct = row['percentage']
        print(f"  {event:<20} {count:>10,}건 ({pct:>5.2f}%)")
    
    # CFR
    death_count = filtered.filter(pl.col('event_type') == 'Death').select(pl.len()).collect().item()
    cfr = (death_count / total * 100) if total > 0 else 0
    print(f"\n  ⚠️  치명률(CFR): {cfr:.2f}%")
    
    # ==========================================
    # 2. 환자 피해 (patient_harm)
    # ==========================================
    print(f"\n2.  환자 피해 (Patient Harm):")
    print("-" * 120)
    
    harm_count = filtered.filter(pl.col('patient_harm').is_not_null()).select(pl.len()).collect().item()
    harm_pct = (harm_count / total * 100)
    
    print(f"  환자 피해 기록 있음: {harm_count:>10,}건 ({harm_pct:>5.2f}%)")
    
    if harm_count > 0:
        # 환자 피해 내용 샘플 (상위 5개)
        harm_samples = filtered.filter(
            pl.col('patient_harm').is_not_null()
        ).select('patient_harm').head(5).collect()
        
        print(f"\n  ◼️ 환자 피해 샘플 (5건):")
        for i, row in enumerate(harm_samples.iter_rows(named=True), 1):
            harm_text = row['patient_harm'][:100] if row['patient_harm'] else ""
            print(f"    {i}. {harm_text}...")
    
    # ==========================================
    # 3. 문제 기기 부품 (problem_components)
    # ==========================================
    print(f"\n3.  문제 기기 부품 (Problem Components):")
    print("-" * 120)
    
    component_count = filtered.filter(pl.col('problem_components').is_not_null()).select(pl.len()).collect().item()
    component_pct = (component_count / total * 100)
    
    print(f"  문제 부품 기록 있음: {component_count:>10,}건 ({component_pct:>5.2f}%)")
    
    if component_count > 0:
        # Top 5 문제 부품
        component_dist = filtered.filter(
            pl.col('problem_components').is_not_null()
        ).group_by('problem_components').len().sort('len', descending=True).head(5).collect()
        
        print(f"\n  🔧 상위 5개 문제 부품:")
        for i, row in enumerate(component_dist.iter_rows(named=True), 1):
            component_str = str(row) if row['problem_components'] else ""
            count = row['len']
            print(f"    {i}. {component_str:<60} {count:>5,}건")
    
    # ==========================================
    # 4. 결함 확인 여부 (defect_confirmed)
    # ==========================================
    print(f"\n4.  제조사 검사 - 결함 확인 여부 (Defect Confirmed):")
    print("-" * 120)
    
    defect_confirmed_dist = filtered.group_by('defect_confirmed').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    for row in defect_confirmed_dist.iter_rows(named=True):
        confirmed = row['defect_confirmed'] if row['defect_confirmed'] else "(NULL)"
        count = row['count']
        pct = row['percentage']
        print(f"  {confirmed:<30} {count:>10,}건 ({pct:>5.2f}%)")
    
    # ==========================================
    # 5. 결함 종류 (defect_type)
    # ==========================================
    print(f"\n5.  결함 종류 (Defect Type) - Top 10:")
    print("-" * 120)
    
    defect_type_count = filtered.filter(pl.col('defect_type').is_not_null()).select(pl.len()).collect().item()
    
    if defect_type_count > 0:
        defect_type_dist = filtered.filter(
            pl.col('defect_type').is_not_null()
        ).group_by('defect_type').agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / defect_type_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).head(10).collect()
        
        for i, row in enumerate(defect_type_dist.iter_rows(named=True), 1):
            defect = row['defect_type'][:60] if row['defect_type'] else ""
            count = row['count']
            pct = row['percentage']
            print(f"  {i:2d}. {defect:<60} {count:>8,}건 ({pct:>5.2f}%)")
    else:
        print(f"  결함 종류 기록 없음")
    
    # ==========================================
    # 6. 검사 조치 (inspection_actions)
    # ==========================================
    print(f"\n6.  제조사 검사 조치 (Inspection Actions):")
    print("-" * 120)
    
    inspection_count = filtered.filter(pl.col('inspection_actions').is_not_null()).select(pl.len()).collect().item()
    inspection_pct = (inspection_count / total * 100)
    
    print(f"  검사 조치 기록 있음: {inspection_count:>10,}건 ({inspection_pct:>5.2f}%)")
    
    if inspection_count > 0:
        # 검사 조치 샘플 (상위 3개)
        inspection_samples = filtered.filter(
            pl.col('inspection_actions').is_not_null()
        ).select('inspection_actions').collect()
        
        print(f"\n  ◼️ 검사 조치 샘플 (3건):")
        for i, row in enumerate(inspection_samples.iter_rows(named=True), 1):
            action_text = row['inspection_actions'][:100] if row['inspection_actions'] else ""
            print(f"    {i}. {action_text}...")
    
    # ==========================================
    # 7. 당시 상황 (incident_summary)
    # ==========================================
    print(f"\n7.  사건 당시 상황 (Incident Summary):")
    print("-" * 120)
    
    summary_count = filtered.filter(pl.col('incident_summary').is_not_null()).select(pl.len()).collect().item()
    summary_pct = (summary_count / total * 100)
    
    print(f"  사건 요약 기록 있음: {summary_count:>10,}건 ({summary_pct:>5.2f}%)")
    
    if summary_count > 0:
        # 사건 요약 샘플 (상위 3개)
        summary_samples = filtered.filter(
            pl.col('incident_summary').is_not_null()
        ).select('incident_summary').collect()
        
        print(f"\n  ◼️ 사건 요약 샘플 (3건):")
        for i, row in enumerate(summary_samples.iter_rows(named=True), 1):
            summary_text = row['incident_summary'][:150] if row['incident_summary'] else ""
            print(f"    {i}. {summary_text}...")
    
    print("\n" + "=" * 120)
    
    return {
        'total': total,
        'cfr': cfr,
        'event_dist': event_dist,
        'harm_count': harm_count,
        'component_count': component_count,
        'defect_confirmed_dist': defect_confirmed_dist,
        'defect_type_count': defect_type_count,
        'inspection_count': inspection_count,
        'summary_count': summary_count
    }

In [39]:
top_n = top_10_models.select("manufacturer_model").item(1, 0)
# top1이 "제조사 - 모델" 형태니까 split해서 넣기
mfr, model = top_n.split(" - ", 1)

result = analyze_manufacturer_model_with_derived(lf, mfr, model,
                                                 manufacturer_col="manufacturer",
                                                 model_col="udi_di")

제조사-모델 종합 분석: ST JUDE MEDICAL - 05415067017246

◼️ 총 보고 건수: 372건

1.  사건 유형 분포:
------------------------------------------------------------------------------------------------------------------------
  Injury                      371건 (99.73%)
  Malfunction                   1건 ( 0.27%)

  ⚠️  치명률(CFR): 0.00%

2.  환자 피해 (Patient Harm):
------------------------------------------------------------------------------------------------------------------------
  환자 피해 기록 있음:        368건 (98.92%)

  ◼️ 환자 피해 샘플 (5건):
    1. Serious Injury...
    2. Serious Injury...
    3. Serious Injury...
    4. Minor Injury...
    5. Serious Injury...

3.  문제 기기 부품 (Problem Components):
------------------------------------------------------------------------------------------------------------------------
  문제 부품 기록 있음:        368건 (98.92%)

  🔧 상위 5개 문제 부품:
    1. {'problem_components': ['lead'], 'len': 170}                   170건
    2. {'problem_components': ['leads'], 'len': 42}                    42건

### Top N 제조사-모델의 파생 컬럼 비교 

In [40]:
def compare_top_models_derived_features(
    lf,
    top_n: int = 5,
    manufacturer_col: str = "manufacturer",
    model_col: str = "udi_di",
):
    """
    상위 N개 제조사-모델(조합)별로 파생 컬럼들의 '기록 비율(coverage)'과 CFR을 한 줄 요약으로 비교 출력한다.

    동작 
    ---------
    1) manufacturer_col과 model_col을 결합하여 `manufacturer_model` 조합 컬럼을 만든다.
       - 제조사/모델이 모두 있으면: "제조사 - 모델"
       - 그 외: "(정보 없음)"
    2) 조합별 보고 건수(total)를 집계하여 상위 top_n개 조합을 선정한다.
    3) 각 조합에 대해 아래 지표를 계산하여 한 줄로 출력한다.
       - CFR: Death 비율(Death / total)
       - 환자피해(patient_harm) 기록률: patient_harm가 null이 아닌 비율
       - 문제부품(problem_components) 기록률
       - 결함확인(defect_confirmed) 기록률
       - 결함종류(defect_type) 기록률
       - 검사조치(inspection_actions) 기록률

    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임 (파생 컬럼 포함)
    top_n : int, default=5
        비교할 상위 제조사-모델 조합 수
    manufacturer_col : str, default="manufacturer"
        제조사 컬럼명
    model_col : str, default="device_0_model_number"
        모델/모델번호 컬럼명

    Returns
    -------
    None
        콘솔 출력용 함수(반환값 없음)

    Notes
    -----
    - 문자열 리터럴은 반드시 `pl.lit()`로 감싸 LazyFrame 최적화 과정에서 문자열이 컬럼명으로 오해되는 에러를 방지한다.
    - `.len()` 대신 `.agg(pl.len().alias("total"))`을 사용하여 컬럼명('len') 의존 문제를 피한다.
    - CFR 계산 시 event_type의 대소문자/공백 변형을 고려해 strip + uppercase 후 "DEATH"로 비교한다.

    Examples
    --------
    >>> # Top 5 모델의 파생 컬럼 기록률(coverage) 비교
    >>> compare_top_models_derived_features(lf, top_n=5)

    >>> # 모델 컬럼을 브랜드명으로 바꿔 비교(해당 컬럼이 있을 때)
    >>> compare_top_models_derived_features(lf, top_n=5,
    ...                                    manufacturer_col="manufacturer",
    ...                                    model_col="udi_di")
    """
    # lf = lf.lazy() if isinstance(lf, pl.DataFrame) else lf

    # 조합 컬럼 생성 (리터럴은 pl.lit)
    combo_expr = (
        pl.when(pl.col(manufacturer_col).is_not_null() & pl.col(model_col).is_not_null())
          .then(
              pl.col(manufacturer_col).cast(pl.Utf8)
              + pl.lit(" - ")
              + pl.col(model_col).cast(pl.Utf8)
          )
          .otherwise(pl.lit("(정보 없음)"))
          .alias("manufacturer_model")
    )

    # 상위 N개 선정
    top_models = (
        lf.with_columns(combo_expr)
          .group_by("manufacturer_model")
          .agg(pl.len().alias("total"))
          .sort(pl.col("total"), descending=True)
          .head(top_n)
          .collect()
    )

    print("=" * 140)
    print(f"Top {top_n} 제조사-모델별 파생 컬럼 비교")
    print("=" * 140)
    print(f"{'순위':<4} {'제조사-모델':<45} {'총건수':>8} {'CFR':>6} {'환자피해':>8} {'문제부품':>8} {'결함확인':>8} {'결함종류':>8} {'검사조치':>8}")
    print("-" * 140)

    lf_with_combo = lf.with_columns(combo_expr)

    def pct(n, d):
        return (n / d * 100) if d else 0.0

    for rank, row in enumerate(top_models.iter_rows(named=True), 1):
        mfr_model = row["manufacturer_model"]
        total = row["total"]

        model_data = lf_with_combo.filter(pl.col("manufacturer_model") == pl.lit(mfr_model))

        # CFR (Death)
        death_count = (
            model_data.filter(
                pl.col("event_type").cast(pl.Utf8).str.strip_chars().str.to_uppercase() == "DEATH"
            )
            .select(pl.len())
            .collect()
            .item()
        )
        cfr = pct(death_count, total)

        # 파생 컬럼 기록률(coverage)
        harm_cnt = model_data.filter(pl.col("patient_harm").is_not_null()).select(pl.len()).collect().item()
        component_cnt = model_data.filter(pl.col("problem_components").is_not_null()).select(pl.len()).collect().item()
        defect_confirmed_cnt = model_data.filter(pl.col("defect_confirmed").is_not_null()).select(pl.len()).collect().item()
        defect_type_cnt = model_data.filter(pl.col("defect_type").is_not_null()).select(pl.len()).collect().item()
        inspection_cnt = model_data.filter(pl.col("inspection_actions").is_not_null()).select(pl.len()).collect().item()

        model_short = mfr_model[:43] if len(mfr_model) > 43 else mfr_model
        print(
            f"{rank:<4} {model_short:<45} {total:>8,} "
            f"{cfr:>5.1f}% {pct(harm_cnt,total):>7.1f}% {pct(component_cnt,total):>7.1f}% "
            f"{pct(defect_confirmed_cnt,total):>7.1f}% {pct(defect_type_cnt,total):>7.1f}% {pct(inspection_cnt,total):>7.1f}%"
        )

    print("=" * 140)
    print("\n해석:")
    print("  - CFR: 치명률 (Death / 전체)")
    print("  - 환자피해~검사조치: 각 파생 컬럼에 데이터가 있는 비율(coverage)")


In [41]:
compare_top_models_derived_features(lf, top_n=5)

Top 5 제조사-모델별 파생 컬럼 비교
순위   제조사-모델                                             총건수    CFR     환자피해     문제부품     결함확인     결함종류     검사조치
--------------------------------------------------------------------------------------------------------------------------------------------
1    MEDTRONIC - 00643169633759                         406   0.2%    99.8%    99.8%    94.6%    99.8%    99.8%
2    ST JUDE MEDICAL - 05415067017246                   372   0.0%    98.9%    98.9%    98.7%    98.9%    98.9%
3    ABBOTT VASCULAR - 08717648235184                   336   0.0%   100.0%   100.0%    99.4%   100.0%   100.0%
4    ABIOMED - 00813502011876                           309   9.4%   100.0%   100.0%    97.7%    99.7%   100.0%
5    ST JUDE MEDICAL - 05415067031419                   260   0.0%    99.2%    99.2%    95.8%    99.2%    99.2%

해석:
  - CFR: 치명률 (Death / 전체)
  - 환자피해~검사조치: 각 파생 컬럼에 데이터가 있는 비율(coverage)


### 결함 종류별 제조사-모델 분포 (특정 고정 타입으로 분석 함수)

In [42]:
def analyze_defect_by_manufacturer_model(
    lf,
    defect_type: str,
    top_n: int = 10,
    manufacturer_col: str = "manufacturer",
    model_col: str = "udi_di",
):
    """
    특정 결함 종류(defect_type)가 발생한 보고서들 중, 제조사-모델(조합) Top N을 집계해 출력/반환한다.

    동작
    ---------
    1) defect_type에 해당하는 행만 필터링한다.
    2) (제조사 - 모델) 형태의 조합 문자열 컬럼 `manufacturer_model`을 생성한다.
       - 제조사/모델이 모두 있으면: "제조사 - 모델"
       - 그 외: "(정보 없음)"
    3) manufacturer_model별 발생 건수(count)와 전체 대비 비율(percentage)을 계산한다.
    4) 상위 top_n개를 출력하고, 결과 DataFrame을 반환한다.

    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    defect_type : str
        분석할 결함 종류(라벨)
    top_n : int, default=10
        상위 N개 제조사-모델 조합을 출력
    manufacturer_col : str, default="manufacturer"
        제조사 컬럼명
    model_col : str, default="device_0_model_number"
        모델(또는 모델번호) 컬럼명

    Returns
    -------
    polars.DataFrame | None
        데이터가 있으면 제조사-모델별 집계 결과를 반환한다.
        - manufacturer_model : str
        - count : int
        - percentage : float
        해당 결함 데이터가 없으면 None 반환.

    Examples
    --------
    >>> # "Software Failure" 결함이 많이 발생한 제조사-모델 Top 10
    >>> analyze_defect_by_manufacturer_model(lf, "Software Failure", top_n=10)

    >>> # 모델 컬럼을 브랜드명으로 바꿔 분석
    >>> analyze_defect_by_manufacturer_model(
    ...     lf, "Software Failure", top_n=10,
    ...     manufacturer_col="manufacturer",
    ...     model_col="udi_di"
    ... )
    """
    # lf = lf.lazy() if isinstance(lf, pl.DataFrame) else lf

    # defect_type 정규화 비교(공백/대소문자 문제 방지)
    defect_data = lf.filter(
        pl.col("defect_type").cast(pl.Utf8).str.strip_chars().str.to_uppercase()
        == pl.lit(defect_type).cast(pl.Utf8).str.strip_chars().str.to_uppercase()
    )

    total = defect_data.select(pl.len()).collect().item()
    if total == 0:
        print(f"⚠️  '{defect_type}' 결함 데이터가 없습니다.")
        return None

    combo_expr = (
        pl.when(pl.col(manufacturer_col).is_not_null() & pl.col(model_col).is_not_null())
          .then(
              pl.col(manufacturer_col).cast(pl.Utf8)
              + pl.lit(" - ")
              + pl.col(model_col).cast(pl.Utf8)
          )
          .otherwise(pl.lit("(정보 없음)"))
          .alias("manufacturer_model")
    )

    result = (
        defect_data.with_columns(combo_expr)
        .group_by("manufacturer_model")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / total * 100).round(2).alias("percentage"))
        .sort(pl.col("count"), descending=True)
        .head(top_n)
        .collect()
    )

    print("=" * 100)
    print(f"결함 종류: {defect_type}")
    print(f"해당 결함 발생 제조사-모델 Top {top_n} (전체: {total:,}건)")
    print("=" * 100)
    print(f"{'순위':>4} {'제조사 - 모델':<60} {'발생건수':>12} {'비율':>12}")
    print("-" * 100)

    for i, row in enumerate(result.iter_rows(named=True), 1):
        name = (row["manufacturer_model"] or "").replace("\n", " ")
        name = name[:58] if len(name) > 58 else name
        print(f"{i:4d} {name:<60} {row['count']:>12,} {row['percentage']:>11.2f}%")

    print("=" * 100)
    return result


In [43]:
# "Functional Failure"가 많이 발생한 모델 Top 10
analyze_defect_by_manufacturer_model(lf, 'Functional Failure', top_n=10)

결함 종류: Functional Failure
해당 결함 발생 제조사-모델 Top 10 (전체: 935건)
  순위 제조사 - 모델                                                             발생건수           비율
----------------------------------------------------------------------------------------------------
   1 ABIOMED - 00813502011876                                               54        5.78%
   2 ST JUDE MEDICAL - 05415067017246                                       48        5.13%
   3 THORATEC - 00813024013297                                              30        3.21%
   4 ABIOMED - 00813502012828                                               29        3.10%
   5 ST JUDE MEDICAL - 05414734502894                                       28        2.99%
   6 BOSTON SCIENTIFIC NEUROMODULATION - 08714729951254                     24        2.57%
   7 MEDTRONIC - 00643169781702                                             19        2.03%
   8 MEDTRONIC - 00763000315467                                             19        2.03%
   9 BOSTON

manufacturer_model,count,percentage
str,u32,f64
"""ABIOMED - 00813502011876""",54,5.78
"""ST JUDE MEDICAL - 054150670172…",48,5.13
"""THORATEC - 00813024013297""",30,3.21
"""ABIOMED - 00813502012828""",29,3.1
"""ST JUDE MEDICAL - 054147345028…",28,2.99
"""BOSTON SCIENTIFIC NEUROMODULAT…",24,2.57
"""MEDTRONIC - 00643169781702""",19,2.03
"""MEDTRONIC - 00763000315467""",19,2.03
"""BOSTON SCIENTIFIC NEUROMODULAT…",19,2.03


# 재활용 기기 / 일회용 기기

### 일회용/재사용 기기의 피해 확인

- 단일

In [44]:
def analyze_single_column_pretty(lf, column_name, column_display_name=None):
    """
    단일 컬럼의 분포를 예쁘게 출력하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    column_display_name : str, optional
        출력 시 표시할 컬럼 이름 (None이면 column_name 사용)
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼값, count, percentage 포함)
    
    Examples
    --------
    >>> # Reprocessed Flag 분포
    >>> analyze_single_column_pretty(lf, 'reprocessed_and_reused_flag')
    
    >>> # Event Type 분포
    >>> analyze_single_column_pretty(
    ...     lf, 
    ...     'event_type',
    ...     column_display_name='사건 유형'
    ... )
    """
    # 전체 개수
    total = lf.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = lf.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    # 표시 이름
    display_name = column_display_name if column_display_name else column_name
    
    # 출력
    print("=" * 70)
    print(f"{display_name} 분포 (전체: {total:,}건)")
    print("=" * 70)
    print(f"{'값':<20} {'사건 수':>15} {'비율':>15}")
    print("-" * 70)
    
    for row in dist.iter_rows(named=True):
        value = row[column_name] if row[column_name] else "(NULL)"
        count = row['count']
        pct = row['percentage']
        
        # 긴 값은 자르기
        display_value = value[:18] if len(str(value)) > 18 else value
        print(f"{display_value:<20} {count:>15,} {pct:>14.2f}%")
    
    print("=" * 70)
    
    return dist

In [45]:
reuse_dist = analyze_single_column_pretty(
    lf, 
    'reprocessed_and_reused_flag',
    column_display_name='재처리/재사용 여부'
)

재처리/재사용 여부 분포 (전체: 10,000건)
값                               사건 수              비율
----------------------------------------------------------------------
(NULL)                         9,914          99.14%
(NULL)                            82           0.82%
1                                  4           0.04%


- 두 컬럼 크로스탭 분석

In [46]:
def analyze_crosstab(lf, column1, column2, 
                     column1_display_name=None, 
                     column2_display_name=None,
                     show_detail=True):
    """
    두 컬럼의 크로스탭(교차 분석)을 수행하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column1 : str
        첫 번째 컬럼 (예: 'reprocessed_and_reused_flag')
    column2 : str
        두 번째 컬럼 (예: 'event_type')
    column1_display_name : str, optional
        첫 번째 컬럼의 표시 이름
    column2_display_name : str, optional
        두 번째 컬럼의 표시 이름
    show_detail : bool, default=True
        각 column1 값별로 상세 분포를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        크로스탭 결과
    
    Examples
    --------
    >>> # Reuse Flag × Event Type
    >>> analyze_crosstab(
    ...     lf, 
    ...     'reprocessed_and_reused_flag', 
    ...     'event_type'
    ... )
    
    >>> # 제조사 × Event Type
    >>> analyze_crosstab(
    ...     lf,
    ...     'manufacturer',
    ...     'event_type',
    ...     column1_display_name='제조사',
    ...     column2_display_name='사건 유형'
    ... )
    
    >>> # 크로스탭만 보기 (상세 분포 제외)
    >>> analyze_crosstab(
    ...     lf,
    ...     'reprocessed_and_reused_flag',
    ...     'event_type',
    ...     show_detail=False
    ... )
    """
    # 표시 이름 설정
    col1_name = column1_display_name if column1_display_name else column1
    col2_name = column2_display_name if column2_display_name else column2
    
    # 크로스탭 생성
    crosstab = lf.group_by([column1, column2]).agg([
        pl.len().alias('count')
    ]).sort([column1, 'count'], descending=[False, True]).collect()
    
    print("=" * 90)
    print(f"{col1_name} × {col2_name} 크로스탭")
    print("=" * 90)
    print(crosstab)
    
    # 상세 분포 출력
    if show_detail:
        print("\n" + "=" * 90)
        print(f"{col1_name}별 {col2_name} 상세 분포")
        print("=" * 90)
        
        # column1의 고유값 추출 (NULL 포함)
        unique_values = lf.select(column1).unique().collect()[column1].to_list()
        
        # NULL도 포함
        if None not in unique_values and lf.filter(pl.col(column1).is_null()).select(pl.len()).collect().item() > 0:
            unique_values.append(None)
        
        for value in unique_values:
            value_display = value if value else "(NULL)"
            
            # 해당 값의 총 개수
            value_total = lf.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).select(pl.len()).collect().item()
            
            if value_total == 0:
                continue
            
            print(f"\n[{value_display}] 총 {value_total:,}건")
            print("-" * 90)
            
            # 해당 값의 column2 분포
            dist = lf.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).group_by(column2).agg([
                pl.len().alias('count')
            ]).with_columns([
                (pl.col('count') / value_total * 100).round(2).alias('percentage')
            ]).sort('count', descending=True).collect()
            
            print(f"{col2_name:<25} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for row in dist.iter_rows(named=True):
                cat = row[column2] if row[column2] else "(NULL)"
                count = row['count']
                pct = row['percentage']
                
                cat_display = cat[:23] if len(str(cat)) > 23 else cat
                print(f"{cat_display:<25} {count:>15,} {pct:>14.2f}%")
        
        print("=" * 90)
    
    return crosstab

- 재사용 x 이벤트타입

In [47]:
crosstab_result = analyze_crosstab(
    lf,
    'reprocessed_and_reused_flag',
    'event_type',
    column1_display_name='재사용 여부',
    column2_display_name='사건 유형'
)

재사용 여부 × 사건 유형 크로스탭
shape: (7, 3)
┌─────────────────────────────┬─────────────┬───────┐
│ reprocessed_and_reused_flag ┆ event_type  ┆ count │
│ ---                         ┆ ---         ┆ ---   │
│ bool                        ┆ cat         ┆ u32   │
╞═════════════════════════════╪═════════════╪═══════╡
│ null                        ┆ Injury      ┆ 46    │
│ null                        ┆ Malfunction ┆ 31    │
│ null                        ┆ Death       ┆ 5     │
│ false                       ┆ Injury      ┆ 6168  │
│ false                       ┆ Malfunction ┆ 3490  │
│ false                       ┆ Death       ┆ 256   │
│ true                        ┆ Injury      ┆ 4     │
└─────────────────────────────┴─────────────┴───────┘

재사용 여부별 사건 유형 상세 분포

[(NULL)] 총 82건
------------------------------------------------------------------------------------------
사건 유형                                  건수              비율
------------------------------------------------------------------------------

In [48]:
crosstab_result = analyze_crosstab(
    lf,
    'single_use_flag',
    'event_type',
    column1_display_name='재사용 라벨링',
    column2_display_name='사건 유형'
)

재사용 라벨링 × 사건 유형 크로스탭
shape: (8, 3)
┌─────────────────┬─────────────┬───────┐
│ single_use_flag ┆ event_type  ┆ count │
│ ---             ┆ ---         ┆ ---   │
│ bool            ┆ cat         ┆ u32   │
╞═════════════════╪═════════════╪═══════╡
│ null            ┆ Injury      ┆ 8     │
│ null            ┆ Malfunction ┆ 1     │
│ false           ┆ Malfunction ┆ 173   │
│ false           ┆ Injury      ┆ 8     │
│ false           ┆ Death       ┆ 1     │
│ true            ┆ Injury      ┆ 6202  │
│ true            ┆ Malfunction ┆ 3347  │
│ true            ┆ Death       ┆ 260   │
└─────────────────┴─────────────┴───────┘

재사용 라벨링별 사건 유형 상세 분포

[(NULL)] 총 9건
------------------------------------------------------------------------------------------
사건 유형                                  건수              비율
------------------------------------------------------------------------------------------
Injury                                  8          88.89%
Malfunction                             1 

In [49]:
crosstab_result = analyze_crosstab(
    lf,
    'previous_use_code',
    'event_type',
    column1_display_name='재처리 여부',
    column2_display_name='사건 유형'
)

재처리 여부 × 사건 유형 크로스탭
shape: (9, 3)
┌───────────────────┬─────────────┬───────┐
│ previous_use_code ┆ event_type  ┆ count │
│ ---               ┆ ---         ┆ ---   │
│ bool              ┆ cat         ┆ u32   │
╞═══════════════════╪═════════════╪═══════╡
│ null              ┆ Injury      ┆ 74    │
│ null              ┆ Malfunction ┆ 63    │
│ null              ┆ Death       ┆ 1     │
│ false             ┆ Malfunction ┆ 167   │
│ false             ┆ Injury      ┆ 1     │
│ false             ┆ Death       ┆ 1     │
│ true              ┆ Injury      ┆ 6143  │
│ true              ┆ Malfunction ┆ 3291  │
│ true              ┆ Death       ┆ 259   │
└───────────────────┴─────────────┴───────┘

재처리 여부별 사건 유형 상세 분포

[(NULL)] 총 138건
------------------------------------------------------------------------------------------
사건 유형                                  건수              비율
------------------------------------------------------------------------------------------
Injury                      

# event_type = death 기기 (치명률)

## 치명률

In [50]:
def calculate_cfr_by_device(lf, device_column='manufacturer', 
                            event_column='event_type',
                            top_n=None, min_cases=10):
    """
    기기별 치명률(Case Fatality Rate)을 계산하는 함수
    
    치명률(CFR) = (사망 건수 / 해당 기기 총 보고 건수) × 100
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str, default='manufacturer'
        기기 컬럼명
    event_column : str, default='event_type'
        사건 유형 컬럼명
    top_n : int, optional
        상위 N개 기기만 분석 (None이면 전체)
    min_cases : int, default=10
        최소 보고 건수 (이보다 적은 기기는 제외, 통계적 신뢰도 확보)
    
    Returns
    -------
    polars.DataFrame
        기기별 치명률 결과 (기기명, 총건수, 사망건수, 부상건수, 오작동건수, CFR)
    
    Examples
    --------
    >>> # 전체 기기 CFR
    >>> cfr_result = calculate_cfr_by_device(lf)
    
    >>> # 상위 20개 기기만
    >>> cfr_top20 = calculate_cfr_by_device(lf, top_n=20)
    
    >>> # 최소 100건 이상 보고된 기기만
    >>> cfr_reliable = calculate_cfr_by_device(lf, min_cases=100)
    """
    
    # 기기별 전체 건수와 사건 유형별 건수
    device_stats = lf.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col(event_column) == 'Death').sum().alias('death_count'),
        (pl.col(event_column) == 'Injury').sum().alias('injury_count'),
        (pl.col(event_column) == 'Malfunction').sum().alias('malfunction_count')
    ]).filter(
        pl.col('total_cases') >= min_cases  # 최소 건수 필터
    ).with_columns([
        # CFR 계산
        (pl.col('death_count') / pl.col('total_cases') * 100).round(2).alias('cfr'),
        # 부상률
        (pl.col('injury_count') / pl.col('total_cases') * 100).round(2).alias('injury_rate'),
        # 오작동률
        (pl.col('malfunction_count') / pl.col('total_cases') * 100).round(2).alias('malfunction_rate')
    ]).sort('cfr', descending=True)
    
    # Top N만
    if top_n:
        device_stats = device_stats.head(top_n)
    
    result = device_stats.collect()
    
    # 출력
    print("=" * 120)
    print(f"기기별 치명률(CFR) 분석 (최소 {min_cases}건 이상)")
    print("=" * 120)
    print(f"{'순위':>4} {'기기명':<50} {'총건수':>10} {'사망':>8} {'부상':>8} {'오작동':>10} {'CFR(%)':>10}")
    print("-" * 120)
    
    for i, row in enumerate(result.iter_rows(named=True), 1):
        device = row[device_column] if row[device_column] else "(NULL)"
        total = row['total_cases']
        death = row['death_count']
        injury = row['injury_count']
        mal = row['malfunction_count']
        cfr = row['cfr']
        
        device_short = device[:48] if len(device) > 48 else device
        
        print(f"{i:4d} {device_short:<50} {total:>10,} {death:>8,} {injury:>8,} {mal:>10,} {cfr:>10.2f}%")
    
    print("=" * 120)
    
    # 요약 통계
    print(f"\n◼️ 요약 통계:")
    print(f"  - 분석 기기 수: {len(result):,}개")
    print(f"  - 평균 CFR: {result['cfr'].mean():.2f}%")
    print(f"  - 최대 CFR: {result['cfr'].max():.2f}%")
    print(f"  - 최소 CFR: {result['cfr'].min():.2f}%")
    print(f"  - CFR 중앙값: {result['cfr'].median():.2f}%")
    
    return result

In [51]:
# 최소 10건 이상 보고된 기기의 CFR
cfr_all = calculate_cfr_by_device(lf, top_n=20)

기기별 치명률(CFR) 분석 (최소 10건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 THORATEC                                                  515      128      167        220      24.85%
   2 HEARTWARE                                                  94       18       16         60      19.15%
   3 NEVRO                                                      82       11       71          0      13.41%
   4 ABIOMED                                                   649       52      317        280       8.01%
   5 TERUMO MEDICAL                                             33        1       21         11       3.03%
   6 MEDTRONIC                                               3,094       34    1,339      1,721       1.10%
   7 ST JUDE MEDICAL                                         1,988        9    1,420        559 

In [52]:
# 제조사별 치명률
cfr_by_manufacturer = calculate_cfr_by_device(
    lf, 
    device_column='manufacturer',
    min_cases=50
)

기기별 치명률(CFR) 분석 (최소 50건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 THORATEC                                                  515      128      167        220      24.85%
   2 HEARTWARE                                                  94       18       16         60      19.15%
   3 NEVRO                                                      82       11       71          0      13.41%
   4 ABIOMED                                                   649       52      317        280       8.01%
   5 MEDTRONIC                                               3,094       34    1,339      1,721       1.10%
   6 ST JUDE MEDICAL                                         1,988        9    1,420        559       0.45%
   7 BOSTON SCIENTIFIC                                       1,482        6      969        507 

### CFR 구간별 분포
- generic_name에서 manufacturer_d_name으로 변경

In [53]:
def analyze_cfr_distribution(lf, device_column='manufacturer', min_cases=10):
    """
    CFR 구간별 기기 분포를 분석하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str
        기기 컬럼명
    min_cases : int
        최소 보고 건수
    
    Examples
    --------
    >>> analyze_cfr_distribution(lf)
    """
    # CFR 계산
    cfr_data = lf.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col('event_type') == 'Death').sum().alias('death_count')
    ]).filter(
        pl.col('total_cases') >= min_cases
    ).with_columns([
        (pl.col('death_count') / pl.col('total_cases') * 100).alias('cfr')
    ]).collect()
    
    # CFR 구간별 분류
    cfr_ranges = [
        (0, 1, "매우 낮음 (0-1%)"),
        (1, 3, "낮음 (1-3%)"),
        (3, 5, "보통 (3-5%)"),
        (5, 10, "높음 (5-10%)"),
        (10, 100, "매우 높음 (10%+)")
    ]
    
    print("=" * 80)
    print("CFR 구간별 기기 분포")
    print("=" * 80)
    print(f"{'CFR 구간':<25} {'기기 수':>15} {'비율':>15}")
    print("-" * 80)
    
    total_devices = len(cfr_data)
    
    for min_cfr, max_cfr, label in cfr_ranges:
        count = cfr_data.filter(
            (pl.col('cfr') >= min_cfr) & (pl.col('cfr') < max_cfr)
        ).shape[0]
        
        pct = (count / total_devices * 100) if total_devices > 0 else 0
        print(f"{label:<25} {count:>15,} {pct:>14.1f}%")
    
    print("=" * 80)
    print(f"{'총 기기 수':<25} {total_devices:>15,} {100.0:>14.1f}%")
    print("=" * 80)

In [54]:
# CFR 구간별 기기 분포
analyze_cfr_distribution(lf, min_cases=50)

CFR 구간별 기기 분포
CFR 구간                               기기 수              비율
--------------------------------------------------------------------------------
매우 낮음 (0-1%)                            7           58.3%
낮음 (1-3%)                               1            8.3%
보통 (3-5%)                               0            0.0%
높음 (5-10%)                              1            8.3%
매우 높음 (10%+)                            3           25.0%
총 기기 수                                 12          100.0%


### 특정 기기의 상세 통계 분석

In [55]:
def analyze_device_detail(lf, device_name, device_column='manufacturer'):
    """
    특정 기기의 상세 통계 분석
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_name : str
        분석할 기기명
    device_column : str
        기기 컬럼명
    
    Examples
    --------
    >>> analyze_device_detail(lf, 'Catheter, Intravascular, Therapeutic')
    """
    device_data = lf.filter(pl.col(device_column) == device_name).collect()
    
    total = len(device_data)
    death = device_data.filter(pl.col('event_type') == 'Death').shape[0]
    injury = device_data.filter(pl.col('event_type') == 'Injury').shape[0]
    mal = device_data.filter(pl.col('event_type') == 'Malfunction').shape[0]
    
    cfr = (death / total * 100) if total > 0 else 0
    injury_rate = (injury / total * 100) if total > 0 else 0
    mal_rate = (mal / total * 100) if total > 0 else 0
    
    print("=" * 80)
    print(f"기기 상세 분석: {device_name}")
    print("=" * 80)
    print(f"\n◼️ 기초 통계:")
    print(f"  총 보고 건수: {total:,}건")
    print(f"\n◼️ 사건 유형별 분포:")
    print(f"  • 사망 (Death):        {death:>8,}건 ({cfr:>5.2f}%) ⚠️ CFR")
    print(f"  • 부상 (Injury):       {injury:>8,}건 ({injury_rate:>5.2f}%)")
    print(f"  • 오작동 (Malfunction): {mal:>8,}건 ({mal_rate:>5.2f}%)")
    print("=" * 80)

In [56]:
# 특정 기기 상세 분석
analyze_device_detail(lf,'BOSTON SCIENTIFIC CORPORATION')

기기 상세 분석: BOSTON SCIENTIFIC CORPORATION

◼️ 기초 통계:
  총 보고 건수: 0건

◼️ 사건 유형별 분포:
  • 사망 (Death):               0건 ( 0.00%) ⚠️ CFR
  • 부상 (Injury):              0건 ( 0.00%)
  • 오작동 (Malfunction):        0건 ( 0.00%)



**기기 결함 종류 분포**

**결함 별 환자 피해/사건**

**기기 결함별 부품 분포**

**종합결함 확인표**

# device_class (EDA)
* 환자 피해 patient_harm
* 문제 기기 problem_components
* 당시 상황 incident_summary
* 결함 여부 defect_confirmed
* 결함 종류 defect_type
* 기기 실험 요약 inspection_actions

### 이상기기 top_n

In [57]:
# def get_top_n_by_column(lf, group_column, top_n=10, column_display_name=None, verbose=True):
#     """
#     특정 컬럼을 기준으로 상위 N개 항목을 추출하고 비율과 함께 출력하는 함수
    
#     Parameters
#     ----------
#     lf : polars.LazyFrame or polars.DataFrame
#         분석할 데이터프레임
#     group_column : str
#         그룹화할 컬럼명 (예: 'device_0_brand_name')
#     top_n : int, default=10
#         상위 N개 항목 추출
#     column_display_name : str, optional
#         출력 시 표시할 컬럼 이름 (None이면 group_column 사용)
#     verbose : bool, default=True
#         결과를 출력할지 여부
    
#     Returns
#     -------
#     polars.DataFrame
#         상위 N개 항목 (컬럼명, count, percentage 포함)
    
#     Examples
#     --------
#     >>> # 기본 사용 - 상위 20개 기기
#     >>> top_devices = get_top_n_by_column(lf, 'device_0_brand_name', top_n=20)
    
#     >>> # 제조사 상위 10개
#     >>> top_manufacturers = get_top_n_by_column(
#     ...     lf, 
#     ...     'device_0_brand_name', 
#     ...     top_n=10,
#     ...     column_display_name='제조사'
#     ... )
    
#     >>> # 출력 없이 결과만
#     >>> result = get_top_n_by_column(lf, 'device_0_brand_name', top_n=5, verbose=False)
#     """
#     # 전체 개수
#     total = lf.select(pl.len()).collect().item()
    
#     # 상위 N개 추출
#     top_items = lf.group_by(group_column).agg([
#         pl.len().alias('count')
#     ]).with_columns([
#         (pl.col('count') / total * 100).round(2).alias('percentage')
#     ]).sort('count', descending=True).head(top_n).collect()
    
#     # 출력
#     if verbose:
#         display_name = column_display_name if column_display_name else group_column
        
#         print("=" * 90)
#         print(f"{display_name} 사건 수 Top {top_n} (전체: {total:,}건)")
#         print("=" * 90)
#         print()
        
#         for i, row in enumerate(top_items.iter_rows(named=True), 1):
#             value = row[group_column] if row[group_column] else "(NULL)"
#             count = row['count']
#             pct = row['percentage']
            
#             # 긴 값은 70자로 자르기
#             display_value = value[:70] if len(value) > 70 else value
            
#             print(f"{i:2d}. {display_value:70s} {count:>8,}건 ({pct:>5.1f}%)")
    
#     return top_items

In [58]:
# top_devices = get_top_n_by_column(
#     lf, 
#     'device_0_brand_name', 
#     top_n=20
# )

### device 3 기기 event_type 분포 확인

In [59]:
def analyze_top_n_by_category(lf, group_column, category_column, top_n=3, 
                               group_display_name=None, category_display_name=None, 
                               verbose=True):
    """
    상위 N개 항목에 대해 카테고리별 분포를 분석하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    group_column : str
        그룹화할 주요 컬럼 (예: 'device_0_brand_name')
    category_column : str
        분포를 확인할 카테고리 컬럼 (예: 'event_type')
    top_n : int, default=3
        상위 N개 항목 분석
    group_display_name : str, optional
        그룹 컬럼의 표시 이름 (None이면 group_column 사용)
    category_display_name : str, optional
        카테고리 컬럼의 표시 이름 (None이면 category_column 사용)
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    dict
        각 상위 항목별 카테고리 분포 딕셔너리
    
    Examples
    --------
    >>> # Top 3 기기별 Event Type 분포
    >>> result = analyze_top_n_by_category(
    ...     lf, 
    ...     'device_0_brand_name', 
    ...     'event_type',
    ...     top_n=3
    ... )
    
    >>> # Top 5 제조사별 Event Type 분포
    >>> result = device_0_brand_name(
    ...     lf,
    ...     'manufacturer',
    ...     'event_type',
    ...     top_n=5,
    ...     group_display_name='제조사',
    ...     category_display_name='사건 유형'
    ... )
    
    >>> # Top 10 기기별 재처리 여부 분포
    >>> result = device_0_brand_name(
    ...     lf,
    ...     'manufacturer',
    ...     'reprocessed_and_reused_flag',
    ...     top_n=10,
    ...     category_display_name='재처리 여부'
    ... )
    """
    # 상위 N개 항목 추출
    top_items = lf.group_by(group_column).len()\
        .sort('len', descending=True).head(top_n).collect()
    
    # 결과 저장용 딕셔너리
    results = {}
    
    # 표시 이름 설정
    group_name = group_display_name if group_display_name else group_column
    category_name = category_display_name if category_display_name else category_column
    
    if verbose:
        print("=" * 90)
        print(f"Top {top_n} {group_name}별 {category_name} 분포")
        print("=" * 90)
    
    # 각 항목에 대해 카테고리 분포 계산
    for rank, row in enumerate(top_items.iter_rows(named=True), 1):
        item_value = row[group_column]
        total_count = row['len']
        
        # 해당 항목의 카테고리 분포
        dist = lf.filter(
            pl.col(group_column) == item_value
        ).group_by(category_column).agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / total_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).collect()
        
        # 결과 저장
        results[item_value] = dist
        
        # 출력
        if verbose:
            print(f"\n[{rank}위] {item_value}")
            print(f"총 사건 수: {total_count:,}건")
            print("-" * 90)
            print(f"{category_name:<20} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for cat_row in dist.iter_rows(named=True):
                category = cat_row[category_column] if cat_row[category_column] else "(NULL)"
                count = cat_row['count']
                pct = cat_row['percentage']
                print(f"{category:<20} {count:>15,} {pct:>14.1f}%")
    
    if verbose:
        print("=" * 90)
    
    return results

In [60]:
result = analyze_top_n_by_category(
    lf,
    'manufacturer',
    'event_type',
    top_n=3
)

Top 3 manufacturer별 event_type 분포

[1위] MEDTRONIC
총 사건 수: 3,094건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Malfunction                    1,721           55.6%
Injury                         1,339           43.3%
Death                             34            1.1%

[2위] ST JUDE MEDICAL
총 사건 수: 1,988건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Injury                         1,420           71.4%
Malfunction                      559           28.1%
Death                              9            0.5%

[3위] BOSTON SCIENTIFIC
총 사건 수: 1,482건
-----------------------------------------------------------------------

### 결함 종류별 문제 기기 부품 분석

In [61]:
def analyze_defect_components(lf, defect_type, top_n=10):
    """
    특정 결함 종류의 문제 기기 부품 분석
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    defect_type : str
        분석할 결함 종류
    top_n : int, default=10
        상위 N개 문제 부품 표시
    
    Returns
    -------
    polars.DataFrame
        문제 부품 분포
    
    Examples
    --------
    >>> # 특정 결함의 문제 부품
    >>> analyze_defect_components(lf, 'Software Failure', top_n=15)
    """
    # 해당 결함 필터링
    defect_data = lf.filter(
        pl.col('defect_type') == defect_type
    )
    
    total = defect_data.select(pl.len()).collect().item()
    
    # 문제 부품 분포
    component_dist = defect_data.group_by('problem_components').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    print("=" * 100)
    print(f"결함 종류: {defect_type}")
    print(f"문제 기기 부품 분석 (전체: {total:,}건)")
    print("=" * 100)
    print(f"{'순위':>4} {'문제 부품':<60} {'건수':>12} {'비율':>12}")
    print("-" * 100)
    
    for i, row in enumerate(component_dist.iter_rows(named=True), 1):
        component = row['problem_components']
        count = row['count']
        pct = row['percentage']
        
        # 리스트를 문자열로 변환한 후 처리
        component_str = str(component) if component else "(NULL)"
        component_display = component_str[:58] if component_str and len(component_str) > 58 else (component_str or "(NULL)")
        print(f"{i:4d} {component_display:<60} {count:>12,} {pct:>11.2f}%")
    
    print("=" * 100)
    
    return component_dist

In [62]:
# "Sensor/Accuracy" 결함의 문제 부품
components = analyze_defect_components(lf, 'Sensor/Accuracy', top_n=15)

결함 종류: Sensor/Accuracy
문제 기기 부품 분석 (전체: 2,103건)
  순위 문제 부품                                                                  건수           비율
----------------------------------------------------------------------------------------------------
   1 ['lead']                                                              549       26.11%
   2 ['rv lead']                                                           243       11.55%
   3 ['ra lead']                                                            65        3.09%
   4 ['ra lead', 'rv lead']                                                 60        2.85%
   5 ['rv lead', 'ra lead']                                                 53        2.52%
   6 ['pacemaker']                                                          27        1.28%
   7 ['lead', 'ipg']                                                        26        1.24%
   8 ['leads']                                                              23        1.09%
   9 ['lead', 'device']

### 분포도 확인 함수 코드

In [63]:
def analyze_column_distribution(lf, column_name, top_n=None, show_null=True, verbose=True):
    """
    특정 컬럼의 값 분포와 비율을 분석하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    top_n : int, optional
        상위 N개만 표시 (None이면 전체 표시)
    show_null : bool, default=True
        NULL 값 개수를 별도로 표시할지 여부
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼명, count, percentage 포함)
    
    Examples
    --------
    >>> # 기본 사용
    >>> result = analyze_column_distribution(lf, 'event_type')
    
    >>> # 상위 10개만
    >>> result = analyze_column_distribution(lf, 'manufacturer', top_n=10)
    
    >>> # 출력 없이 결과만
    >>> result = analyze_column_distribution(lf, 'event_type', verbose=False)
    """
    # 전체 개수
    total = lf.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = lf.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True)
    
    # 상위 N개만
    if top_n is not None:
        dist = dist.head(top_n)
    
    result = dist
    
    # NULL 개수 확인
    null_count = lf.filter(pl.col(column_name).is_null()).select(pl.len()).collect().item()
    
    # 출력
    if verbose:
        print("=" * 80)
        print(f"{column_name} 분포 (전체: {total:,}건)")
        if top_n:
            print(f"(상위 {top_n}개만 표시)")
        print("=" * 80)
        print(result)
        
        if show_null:
            print(f"\nNULL 값: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    return result

In [64]:
# event_type 분포 확인
event_dist = analyze_column_distribution(lf, 'event_type')
print(event_dist)

event_type 분포 (전체: 10,000건)
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SORT BY [descending: [true]] [col("count")]
   WITH_COLUMNS:
   [[([(col("count")) / (10000)]) * (100.0)].round().alias("percentage")] 
    AGGREGATE[maintain_order: false]
      [len().alias("count")] BY [col("event_type")]
      FROM
      Parquet SCAN [/Users/t2023-m0030/Desktop/i_hate_maude/notebooks/maude_10000_2025-12-19_2.parquet]
      PROJECT */26 COLUMNS
      ESTIMATED ROWS: 10000

NULL 값: 0개 (0.00%)
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SORT BY [descending: [true]] [col("count")]
   WITH_COLUMNS:
   [[([(col("count")) / (10000)]) * (100.0)].round().alias("percentage")] 
    AGGREGATE[maintain_order: false]
      [len().alias("count")] BY [col("event_type")]
      FROM
      Parquet SCAN [/Users/t2023-m0030/Desktop/i_hate_maude/notebooks/maude_10000_2025-12-19_2.parquet]
      PROJECT */26 COLUMNS
      ESTIMATED ROWS: 10000


### 기기 결함 종류 분포 분석

In [65]:
def analyze_defect_types(lf, top_n=20, min_cases=5):
    """
    기기 결함 종류 분포를 분석하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_n : int, default=20
        상위 N개 결함 종류 표시 
    >>> # 전체 결함 종류 분포
    >>> defect_dist = analyze_defect_types(lf)
    
    >>> # 상위 30개만
    >>> defect_top30 = analyze_defect_types(lf, top_n=30)
    """
    # 전체 개수
    total = lf.select(pl.len()).collect().item()
    
    # 결함 종류 분포
    defect_dist = lf.group_by('defect_type').agg([
        pl.len().alias('count')
    ]).filter(
        pl.col('count') >= min_cases
    ).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    # 결함 확인 여부
    defect_confirmed = lf.group_by('defect_confirmed').len().collect()
    
    print("=" * 100)
    print(f"기기 결함 종류 분석 (전체: {total:,}건)")
    print("=" * 100)
    
    # 결함 확인 여부 먼저 출력
    print("\n◼️ 결함 확인 여부:")
    print("-" * 100)
    for row in defect_confirmed.iter_rows(named=True):
        confirmed = row['defect_confirmed']
        count = row['len']
        pct = (count / total * 100)
        confirmed_display = confirmed if confirmed else "(NULL)"
        print(f"  {confirmed_display:<20} {count:>10,}건 ({pct:>5.2f}%)")
    
    # 결함 종류 분포
    print(f"\n◼️ 결함 종류 Top {top_n} (최소 {min_cases}건 이상):")
    print("-" * 100)
    print(f"{'순위':>4} {'결함 종류':<60} {'건수':>12} {'비율':>12}")
    print("-" * 100)
    
    for i, row in enumerate(defect_dist.iter_rows(named=True), 1):
        defect_type = row['defect_type']
        count = row['count']
        pct = row['percentage']
        
        defect_display = defect_type[:58] if defect_type and len(defect_type) > 58 else (defect_type or "(NULL)")
        print(f"{i:4d} {defect_display:<60} {count:>12,} {pct:>11.2f}%")
    
    print("=" * 100)
    
    return defect_dist

In [66]:
# 결함 종류 Top 20
defect_distribution = analyze_defect_types(lf, top_n=20)

기기 결함 종류 분석 (전체: 10,000건)

◼️ 결함 확인 여부:
----------------------------------------------------------------------------------------------------
  1                         5,715건 (57.15%)
  (NULL)                      311건 ( 3.11%)
  (NULL)                    3,974건 (39.74%)

◼️ 결함 종류 Top 20 (최소 5건 이상):
----------------------------------------------------------------------------------------------------
  순위 결함 종류                                                                  건수           비율
----------------------------------------------------------------------------------------------------
   1 Other                                                               2,465       24.65%
   2 Sensor/Accuracy                                                     2,103       21.03%
   3 Unknown                                                             1,599       15.99%
   4 Mechanical/Structural                                               1,404       14.04%
   5 Electrical/Power               

### 결함 종류별 환자 피해/사건 유형 분석

In [67]:
def analyze_defect_impact(lf, top_defects=10):
    """
    주요 결함 종류별 환자 피해 및 사건 유형 분석
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_defects : int, default=10
        분석할 상위 결함 종류 개수
    
    Returns
    -------
    dict
        결함 종류별 분석 결과
    
    Examples
    --------
    >>> # 상위 10개 결함의 영향 분석
    >>> impact_result = analyze_defect_impact(lf, top_defects=10)
    
    >>> # 상위 5개만
    >>> impact_top5 = analyze_defect_impact(lf, top_defects=5)
    """
    # 상위 결함 종류 추출
    top_defects_list = lf.group_by('defect_type').len()\
        .sort('len', descending=True).head(top_defects).collect()
    
    results = {}
    
    print("=" * 110)
    print(f"상위 {top_defects}개 결함 종류별 영향 분석")
    print("=" * 110)
    
    for rank, row in enumerate(top_defects_list.iter_rows(named=True), 1):
        defect_type = row['defect_type']
        total_count = row['len']
        
        if not defect_type:
            defect_type = "(NULL)"
        
        # 해당 결함의 데이터 필터링
        defect_data = lf.filter(
            pl.col('defect_type') == defect_type if defect_type != "(NULL)" 
            else pl.col('defect_type').is_null()
        )
        
        # Event Type 분포
        event_dist = defect_data.group_by('event_type').agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / total_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).collect()
        
        # 환자 피해 여부 (NULL이 아닌 경우)
        harm_count = defect_data.filter(
            pl.col('patient_harm').is_not_null()
        ).select(pl.len()).collect().item()
        
        # CFR 계산
        death_count = defect_data.filter(pl.col('event_type') == 'Death').select(pl.len()).collect().item()
        cfr = (death_count / total_count * 100) if total_count > 0 else 0
        
        results[defect_type] = {
            'total': total_count,
            'event_dist': event_dist,
            'harm_count': harm_count,
            'cfr': cfr
        }
        
        # 출력
        print(f"\n[{rank}위] {defect_type}")
        print(f"총 {total_count:,}건 | 환자 피해 기록: {harm_count:,}건 | CFR: {cfr:.2f}%")
        print("-" * 110)
        print(f"{'사건 유형':<20} {'건수':>15} {'비율':>15}")
        print("-" * 110)
        
        for event_row in event_dist.iter_rows(named=True):
            event = event_row['event_type'] if event_row['event_type'] else "(NULL)"
            count = event_row['count']
            pct = event_row['percentage']
            print(f"{event:<20} {count:>15,} {pct:>14.2f}%")
    
    print("=" * 110)
    
    return results

In [68]:
# 상위 10개 결함의 환자 피해/사건 유형
impact_analysis = analyze_defect_impact(lf, top_defects=10)

상위 10개 결함 종류별 영향 분석

[1위] Other
총 2,465건 | 환자 피해 기록: 2,465건 | CFR: 1.91%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Injury                         2,022          82.03%
Malfunction                      396          16.06%
Death                             47           1.91%

[2위] Sensor/Accuracy
총 2,103건 | 환자 피해 기록: 2,103건 | CFR: 0.81%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Malfunction                    1,405          66.81%
Injury                           681          32.38%
Death                             17           0.


[7위] Communication/Connectivity
총 236건 | 환자 피해 기록: 236건 | CFR: 0.00%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Injury                           132          55.93%
Malfunction                      104          44.07%

[8위] User/Human Factor
총 80건 | 환자 피해 기록: 80건 | CFR: 0.00%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Injury                            47          58.75%
Malfunction                       33          41.25%

[9위] (NULL)
총 32건 | 환자 피해 기록: 5건 | CFR: 0.00%
--------------------------------------------------------------

### 종합 결함 분석 대시보드

In [69]:
def comprehensive_defect_analysis(lf):
    """
    결함에 대한 종합적인 분석을 수행하는 함수
    
    Parameters
    ----------
    lf : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    
    Examples
    --------
    >>> comprehensive_defect_analysis(lf)
    """
    total = lf.select(pl.len()).collect().item()
    
    print("=" * 120)
    print("◼️ Class 3 기기 결함 종합 분석 대시보드")
    print("=" * 120)
    
    # 1. 결함 확인 여부
    print("\n◼️ 1. 제조사 검사 - 결함 확인 여부")
    print("-" * 120)
    defect_confirmed = lf.group_by('defect_confirmed').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    for row in defect_confirmed.iter_rows(named=True):
        confirmed = row['defect_confirmed'] or "(NULL)"
        count = row['count']
        pct = row['percentage']
        print(f"  {confirmed:<30} {count:>15,}건 ({pct:>5.2f}%)")
    
    # 2. 결함 종류 Top 10
    print("\n◼️ 2. 결함 종류 Top 10")
    print("-" * 120)
    defect_types = lf.filter(
        pl.col('defect_type').is_not_null()
    ).group_by('defect_type').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(10).collect()
    
    for i, row in enumerate(defect_types.iter_rows(named=True), 1):
        defect = row['defect_type'][:50]
        count = row['count']
        pct = row['percentage']
        print(f"  {i:2d}. {defect:<50} {count:>10,}건 ({pct:>5.2f}%)")
    
    # 3. 문제 부품 Top 10
    print("\n◼️ 3. 문제 기기 부품 Top 10")
    print("-" * 120)
    components = lf.filter(
        pl.col('problem_components').is_not_null()
    ).group_by('problem_components').agg([
        pl.len().alias('count')
    ]).sort('count', descending=True).head(10).collect()
    
    for i, row in enumerate(components.iter_rows(named=True), 1):
        component_str = str(row['problem_components'])[:50] if row['problem_components'] else ""
        count = row['count']
        print(f"  {i:2d}. {component_str:<50} {count:>10,}건")
    
    # 4. 환자 피해 요약
    print("\n◼️  4. 환자 피해 기록")
    print("-" * 120)
    harm_count = lf.filter(
        pl.col('patient_harm').is_not_null()
    ).select(pl.len()).collect().item()
    harm_pct = (harm_count / total * 100)
    
    print(f"  환자 피해 기록 있음: {harm_count:>10,}건 ({harm_pct:>5.2f}%)")
    print(f"  환자 피해 기록 없음: {total - harm_count:>10,}건 ({100 - harm_pct:>5.2f}%)")
    
    # 5. 검사 조치 요약
    print("\n◼️ 5. 제조사 검사 조치")
    print("-" * 120)
    inspection_count = lf.filter(
        pl.col('inspection_actions').is_not_null()
    ).select(pl.len()).collect().item()
    inspection_pct = (inspection_count / total * 100)
    
    print(f"  검사 조치 기록 있음: {inspection_count:>10,}건 ({inspection_pct:>5.2f}%)")
    print(f"  검사 조치 기록 없음: {total - inspection_count:>10,}건 ({100 - inspection_pct:>5.2f}%)")
    
    print("\n" + "=" * 120)

In [70]:
# 전체 결함 분석 한눈에 보기
comprehensive_defect_analysis(lf)

◼️ Class 3 기기 결함 종합 분석 대시보드

◼️ 1. 제조사 검사 - 결함 확인 여부
------------------------------------------------------------------------------------------------------------------------
  1                                        5,715건 (57.15%)
  (NULL)                                   3,974건 (39.74%)
  (NULL)                                     311건 ( 3.11%)

◼️ 2. 결함 종류 Top 10
------------------------------------------------------------------------------------------------------------------------
   1. Other                                                   2,465건 (24.65%)
   2. Sensor/Accuracy                                         2,103건 (21.03%)
   3. Unknown                                                 1,599건 (15.99%)
   4. Mechanical/Structural                                   1,404건 (14.04%)
   5. Electrical/Power                                        1,050건 (10.50%)
   6. Functional Failure                                        935건 ( 9.35%)
   7. Communication/Connectivity        